# Coleta de variáveis por municipio da BA

In [1]:
import requests
import json
import pandas as pd

url_base='https://servicodados.ibge.gov.br/api/v3'
localidade='N6' # municipio
doc_api='https://servicodados.ibge.gov.br/api/docs/agregados?versao=3'#api-bq

In [2]:
base_final=pd.DataFrame()

In [3]:
def process_json_censo_agro(response: 'json', classificacao: str)-> pd.DataFrame:
    subset_base=[]
    var=None
    for each in response:
        for result in each['resultados']:
            # cada result tem um par: classificações , series
            # achar a variável que interessa na classificação
            for c in result['classificacoes']:
                if c['id']==classificacao:
                    var=c['categoria']
                    break

            # formatar a série
            df=pd.DataFrame(result['series'])
            if var:
                df['variavel'] = list(var.values())[0]
            else:
                df['variavel'] = 'Total'

            df['ano']=df['serie'].apply(lambda x: list(x.keys())[0])
            df['value']=df['serie'].apply(lambda x: list(x.values())[0])
            df['cod_municipio']=df['localidade'].apply(lambda x: x['id'])
            df['nome_municipio']=df['localidade'].apply(lambda x: x['nome'])
            df.drop(['localidade', 'serie'], axis=1, inplace=True)
            subset_base.append(df)
            
    subset_base=pd.concat(subset_base)


    subset_base['value']=subset_base['value'].apply(lambda x: float(x) if x!='-' and x!='...' and x!='X'  else 0)
    subset_base=subset_base.pivot(columns='variavel', index=['cod_municipio', 'nome_municipio'], values='value')
    return subset_base

# Censo Agro

## Códigos das cidades da Bahia (todos que começam com 29)

In [4]:
tabela='6906'

In [5]:
cidades=requests.get(url_base+'/agregados/'+tabela+'/localidades/N6')
cidades=pd.DataFrame(cidades.json())

cidades['ba']=cidades['id'].apply(lambda x: True if x.startswith('29') else False)
cidades_ba=cidades[cidades['ba']]

In [6]:
# formatação para recuperação de informação das bases do ibge
cod_cidades_ba=','.join(cidades_ba.id.values)
cod_cidades_ba='[{}]'.format(cod_cidades_ba)

# Indice de diversidade produtiva dos municipios (IDS)

In [7]:
df_diversity=pd.DataFrame()

### Floricultura

* Variável: Valor da venda de produtos de floricultura e/ou plantas ornamentais (Mil Reais) - 10074
    * Classificação Produtos da floricultura 12519

In [8]:
tabela='6952'
variavel='10074'
classificacao='12519'
classes='[all]'

In [9]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                              'classificacao': classificacao+classes
                             })

In [10]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [11]:
df_diversity=subset_base.drop('Total', axis=1)

In [12]:
df_diversity

,variavel,Flores e folhagens para corte,Gramas,Mudas de plantas ornamentais,Mudas e outras formas de propagação (produzidas para plantio),Plantas ornamentais em vaso,"Plantas, flores, folhagens medicinais",Sementes (produzidas para plantio)
cod_municipio,nome_municipio,,,,,,,
2900108,Abaíra (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0
2900207,Abaré (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0
2900306,Acajutiba (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0
2900355,Adustina (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0
2900405,Água Fria (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),383.0,0.0,6.0,0.0,0.0,55.0,0.0
2933406,Wagner (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0
2933455,Wanderley (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
print('Floricultura: ', df_diversity.shape[1])

Floricultura:  7


### Lavouras temporárias e permanentes

* Variável: Valor da produção - 215
    * Produto das lavouras temporárias e permanentes 782

In [14]:
tabela='5457'
variavel='215'
classificacao='782'
classes='[all]'

In [15]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                              'classificacao': classificacao+classes
                             })

In [16]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [17]:
subset_base.drop('Total', axis=1, inplace=True)

In [18]:
print('Lavouras temporárias e permantes: ', subset_base.shape[1])

Lavouras temporárias e permantes:  71


In [19]:
df_diversity=df_diversity.merge(subset_base, left_index=True, right_index=True, how='outer')
df_diversity

,variavel,Flores e folhagens para corte,Gramas,Mudas de plantas ornamentais,Mudas e outras formas de propagação (produzidas para plantio),Plantas ornamentais em vaso,"Plantas, flores, folhagens medicinais",Sementes (produzidas para plantio),Abacate,Abacaxi*,Alfafa fenada,...,Sisal ou agave (fibra),Soja (em grão),Sorgo (em grão),Tangerina,Tomate,Trigo (em grão),Triticale (em grão),Tungue (fruto seco),Urucum (semente),Uva
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2900207,Abaré (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,106.0,0.0,0.0,0.0,0.0,0.0
2900306,Acajutiba (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2900355,Adustina (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2900405,Água Fria (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),383.0,0.0,6.0,0.0,0.0,55.0,0.0,0.0,88.0,0.0,...,0.0,0.0,0.0,107.0,0.0,0.0,0.0,0.0,81.0,0.0
2933406,Wagner (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,0.0,0.0,0.0,0.0,510.0,0.0,0.0,0.0,0.0,0.0
2933455,Wanderley (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Extração vegetal

* Variável: Valor da produção na extração vegetal
    * Produtos da extração vegeral

In [20]:
tabela='6950'
variavel='145'
classificacao='229'
classes='[all]'

In [21]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                              'classificacao': classificacao+classes
                             })

In [22]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [23]:
subset_base.drop('Total', axis=1, inplace=True)

In [24]:
print('Extração vegetal: ', subset_base.shape[1])

Extração vegetal:  52


In [25]:
df_diversity=df_diversity.merge(subset_base, left_index=True, right_index=True, how='outer')
df_diversity

,variavel,Flores e folhagens para corte,Gramas,Mudas de plantas ornamentais,Mudas e outras formas de propagação (produzidas para plantio),Plantas ornamentais em vaso,"Plantas, flores, folhagens medicinais",Sementes (produzidas para plantio),Abacate,Abacaxi*,Alfafa fenada,...,Oiticica (semente),Outros produtos,Palmito_y,Pequi,Piaçava (fibra),Pinhão,Pupunha (coco),Sorva (goma não elástica),Tucumã,Ucuuba (amêndoa)
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2900207,Abaré (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2900306,Acajutiba (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2900355,Adustina (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2900405,Água Fria (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),383.0,0.0,6.0,0.0,0.0,55.0,0.0,0.0,88.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2933406,Wagner (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2933455,Wanderley (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Horticultura

* Variável: Valor da produção da horticultura
    * Produtos da horticultura

In [26]:
tabela='6954'
variavel='10092'
classificacao='228'
classes='[all]'

In [27]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                              'classificacao': classificacao+classes
                             })

In [28]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [29]:
subset_base.drop('Total', axis=1, inplace=True)

In [30]:
df_diversity=df_diversity.merge(subset_base, left_index=True, right_index=True, how='outer')
df_diversity

,variavel,Flores e folhagens para corte,Gramas,Mudas de plantas ornamentais,Mudas e outras formas de propagação (produzidas para plantio)_x,Plantas ornamentais em vaso,"Plantas, flores, folhagens medicinais",Sementes (produzidas para plantio)_x,Abacate,Abacaxi*,Alfafa fenada,...,Pimentão,Quiabo,Rabanete,Repolho,Rúcula,Salsa,Sementes (produzidas para plantio)_y,Taioba,Tomate (estaqueado),Vagem (feijão vagem)
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2900207,Abaré (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2900306,Acajutiba (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2900355,Adustina (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2900405,Água Fria (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,49.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),383.0,0.0,6.0,0.0,0.0,55.0,0.0,0.0,88.0,0.0,...,81.0,7.0,42.0,45.0,77.0,458.0,0.0,0.0,133.0,351.0
2933406,Wagner (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,158.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,196.0,0.0
2933455,Wanderley (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### Aquicultura

* Variável: Valor da produção
    * Tipo de Produtos da Aquicultura

In [31]:
tabela='3940'
variavel='215'
classificacao='654'
classes='[all]'

In [32]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                              'classificacao': classificacao+classes
                             })

In [33]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [34]:
subset_base.drop('Total', axis=1, inplace=True)

In [35]:
df_diversity=df_diversity.merge(subset_base, left_index=True, right_index=True, how='outer')
df_diversity

,variavel,Flores e folhagens para corte,Gramas,Mudas de plantas ornamentais,Mudas e outras formas de propagação (produzidas para plantio)_x,Plantas ornamentais em vaso,"Plantas, flores, folhagens medicinais",Sementes (produzidas para plantio)_x,Abacate,Abacaxi*,Alfafa fenada,...,"Pintado, cachara, cachapira e pintachara, surubim",Pirapitinga,Pirarucu,Sementes de moluscos,"Tambacu, tambatinga",Tambaqui,Tilápia,Traíra e trairão,Truta,Tucunaré
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2900207,Abaré (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0
2900306,Acajutiba (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2900355,Adustina (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2900405,Água Fria (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),383.0,0.0,6.0,0.0,0.0,55.0,0.0,0.0,88.0,0.0,...,45.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0
2933406,Wagner (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2933455,Wanderley (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Produtos de origem animal

* Variável: Valor da produção
    * Tipo de Produtos de origem animal

In [36]:
tabela='74'
variavel='215'
classificacao='80'
classes='[all]'

In [37]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                              'classificacao': classificacao+classes
                             })

In [38]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [39]:
subset_base.drop('Total', axis=1, inplace=True)

In [40]:
df_diversity=df_diversity.merge(subset_base, left_index=True, right_index=True, how='outer')
df_diversity

,variavel,Flores e folhagens para corte,Gramas,Mudas de plantas ornamentais,Mudas e outras formas de propagação (produzidas para plantio)_x,Plantas ornamentais em vaso,"Plantas, flores, folhagens medicinais",Sementes (produzidas para plantio)_x,Abacate,Abacaxi*,Alfafa fenada,...,Tilápia,Traíra e trairão,Truta,Tucunaré,Casulos do bicho-da-seda,Leite,Lã,Mel de abelha,Ovos de codorna,Ovos de galinha
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,255.0,0.0,1.0,0.0,92.0
2900207,Abaré (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,370.0,0.0,0.0,0.0,35.0
2900306,Acajutiba (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,921.0,0.0,20.0,0.0,21.0
2900355,Adustina (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2271.0,0.0,3.0,0.0,1990.0
2900405,Água Fria (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,265.0,0.0,360.0,4.0,6017.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),383.0,0.0,6.0,0.0,0.0,55.0,0.0,0.0,88.0,0.0,...,24.0,0.0,0.0,0.0,0.0,16330.0,0.0,689.0,0.0,4539.0
2933406,Wagner (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,0.0,0.0,0.0,0.0,0.0,463.0,0.0,0.0,0.0,169.0
2933455,Wanderley (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4090.0,0.0,15.0,0.0,293.0


### Muares (valor da venda)

* Variável: Valor da venda de cabeças de muares nos estabelecimentos agropecuários

In [41]:
tabela='6924'
variavel='10027'

In [42]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                          #    'classificacao': classificacao+classes
                             })

In [43]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [44]:
subset_base.rename(columns={'Total': 'Muares'}, inplace=True)

In [45]:
df_diversity=df_diversity.merge(subset_base, left_index=True, right_index=True, how='outer')
df_diversity

,variavel,Flores e folhagens para corte,Gramas,Mudas de plantas ornamentais,Mudas e outras formas de propagação (produzidas para plantio)_x,Plantas ornamentais em vaso,"Plantas, flores, folhagens medicinais",Sementes (produzidas para plantio)_x,Abacate,Abacaxi*,Alfafa fenada,...,Traíra e trairão,Truta,Tucunaré,Casulos do bicho-da-seda,Leite,Lã,Mel de abelha,Ovos de codorna,Ovos de galinha,Muares
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,255.0,0.0,1.0,0.0,92.0,0.0
2900207,Abaré (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,370.0,0.0,0.0,0.0,35.0,0.0
2900306,Acajutiba (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,921.0,0.0,20.0,0.0,21.0,0.0
2900355,Adustina (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2271.0,0.0,3.0,0.0,1990.0,0.0
2900405,Água Fria (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,265.0,0.0,360.0,4.0,6017.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),383.0,0.0,6.0,0.0,0.0,55.0,0.0,0.0,88.0,0.0,...,0.0,0.0,0.0,0.0,16330.0,0.0,689.0,0.0,4539.0,13.0
2933406,Wagner (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,0.0,0.0,0.0,0.0,463.0,0.0,0.0,0.0,169.0,0.0
2933455,Wanderley (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4090.0,0.0,15.0,0.0,293.0,3.0


### Asininos (valor da venda)

* Variável: Valor da venda de cabeças de asininos nos estabelecimentos agropecuários 

In [46]:
tabela='6922'
variavel='10024'

In [47]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                             })

In [48]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [49]:
subset_base.rename(columns={'Total': 'Asininos'}, inplace=True)

In [50]:
df_diversity=df_diversity.merge(subset_base, left_index=True, right_index=True, how='outer')
df_diversity

,variavel,Flores e folhagens para corte,Gramas,Mudas de plantas ornamentais,Mudas e outras formas de propagação (produzidas para plantio)_x,Plantas ornamentais em vaso,"Plantas, flores, folhagens medicinais",Sementes (produzidas para plantio)_x,Abacate,Abacaxi*,Alfafa fenada,...,Truta,Tucunaré,Casulos do bicho-da-seda,Leite,Lã,Mel de abelha,Ovos de codorna,Ovos de galinha,Muares,Asininos
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,255.0,0.0,1.0,0.0,92.0,0.0,0.0
2900207,Abaré (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,370.0,0.0,0.0,0.0,35.0,0.0,1.0
2900306,Acajutiba (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,921.0,0.0,20.0,0.0,21.0,0.0,0.0
2900355,Adustina (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2271.0,0.0,3.0,0.0,1990.0,0.0,0.0
2900405,Água Fria (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,265.0,0.0,360.0,4.0,6017.0,17.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),383.0,0.0,6.0,0.0,0.0,55.0,0.0,0.0,88.0,0.0,...,0.0,0.0,0.0,16330.0,0.0,689.0,0.0,4539.0,13.0,22.0
2933406,Wagner (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,0.0,0.0,0.0,463.0,0.0,0.0,0.0,169.0,0.0,0.0
2933455,Wanderley (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4090.0,0.0,15.0,0.0,293.0,3.0,1.0


### Equinos (valor da venda)

* Variável: Valor da venda de cabeças de equinos nos estabelecimentos agropecuários 

In [51]:
tabela='6920'
variavel='10021'

In [52]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                             })

In [53]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [54]:
subset_base.rename(columns={'Total': 'Equinos'}, inplace=True)

In [55]:
df_diversity=df_diversity.merge(subset_base, left_index=True, right_index=True, how='outer')
df_diversity

,variavel,Flores e folhagens para corte,Gramas,Mudas de plantas ornamentais,Mudas e outras formas de propagação (produzidas para plantio)_x,Plantas ornamentais em vaso,"Plantas, flores, folhagens medicinais",Sementes (produzidas para plantio)_x,Abacate,Abacaxi*,Alfafa fenada,...,Tucunaré,Casulos do bicho-da-seda,Leite,Lã,Mel de abelha,Ovos de codorna,Ovos de galinha,Muares,Asininos,Equinos
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,255.0,0.0,1.0,0.0,92.0,0.0,0.0,0.0
2900207,Abaré (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,370.0,0.0,0.0,0.0,35.0,0.0,1.0,14.0
2900306,Acajutiba (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,921.0,0.0,20.0,0.0,21.0,0.0,0.0,0.0
2900355,Adustina (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2271.0,0.0,3.0,0.0,1990.0,0.0,0.0,83.0
2900405,Água Fria (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,265.0,0.0,360.0,4.0,6017.0,17.0,3.0,113.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),383.0,0.0,6.0,0.0,0.0,55.0,0.0,0.0,88.0,0.0,...,0.0,0.0,16330.0,0.0,689.0,0.0,4539.0,13.0,22.0,1635.0
2933406,Wagner (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,0.0,0.0,463.0,0.0,0.0,0.0,169.0,0.0,0.0,2.0
2933455,Wanderley (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4090.0,0.0,15.0,0.0,293.0,3.0,1.0,129.0


### Bubalinos (valor da venda)

* Variável: Valor da venda de cabeças de bubalinos nos estabelecimentos agropecuários

In [56]:
tabela='6918'
variavel='10017'

In [57]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                             })

In [58]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [59]:
subset_base.rename(columns={'Total': 'Bubalinos'}, inplace=True)

In [60]:
df_diversity=df_diversity.merge(subset_base, left_index=True, right_index=True, how='outer')
df_diversity

,variavel,Flores e folhagens para corte,Gramas,Mudas de plantas ornamentais,Mudas e outras formas de propagação (produzidas para plantio)_x,Plantas ornamentais em vaso,"Plantas, flores, folhagens medicinais",Sementes (produzidas para plantio)_x,Abacate,Abacaxi*,Alfafa fenada,...,Casulos do bicho-da-seda,Leite,Lã,Mel de abelha,Ovos de codorna,Ovos de galinha,Muares,Asininos,Equinos,Bubalinos
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,255.0,0.0,1.0,0.0,92.0,0.0,0.0,0.0,0.0
2900207,Abaré (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,370.0,0.0,0.0,0.0,35.0,0.0,1.0,14.0,0.0
2900306,Acajutiba (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,921.0,0.0,20.0,0.0,21.0,0.0,0.0,0.0,0.0
2900355,Adustina (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2271.0,0.0,3.0,0.0,1990.0,0.0,0.0,83.0,0.0
2900405,Água Fria (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,265.0,0.0,360.0,4.0,6017.0,17.0,3.0,113.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),383.0,0.0,6.0,0.0,0.0,55.0,0.0,0.0,88.0,0.0,...,0.0,16330.0,0.0,689.0,0.0,4539.0,13.0,22.0,1635.0,0.0
2933406,Wagner (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,0.0,463.0,0.0,0.0,0.0,169.0,0.0,0.0,2.0,0.0
2933455,Wanderley (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4090.0,0.0,15.0,0.0,293.0,3.0,1.0,129.0,0.0


### Galinhas (valor da venda)

* Variável: Valor da venda de cabeças de galinhas, galos, frangas, frangos e pintos nos estabelecimentos agropecuários

In [61]:
tabela='6942'
variavel='10018'

In [62]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                             })

In [63]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [64]:
subset_base.rename(columns={'Total': 'Galinhas'}, inplace=True)

In [65]:
df_diversity=df_diversity.merge(subset_base, left_index=True, right_index=True, how='outer')
df_diversity

,variavel,Flores e folhagens para corte,Gramas,Mudas de plantas ornamentais,Mudas e outras formas de propagação (produzidas para plantio)_x,Plantas ornamentais em vaso,"Plantas, flores, folhagens medicinais",Sementes (produzidas para plantio)_x,Abacate,Abacaxi*,Alfafa fenada,...,Leite,Lã,Mel de abelha,Ovos de codorna,Ovos de galinha,Muares,Asininos,Equinos,Bubalinos,Galinhas
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,255.0,0.0,1.0,0.0,92.0,0.0,0.0,0.0,0.0,27.0
2900207,Abaré (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,370.0,0.0,0.0,0.0,35.0,0.0,1.0,14.0,0.0,54.0
2900306,Acajutiba (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,921.0,0.0,20.0,0.0,21.0,0.0,0.0,0.0,0.0,35.0
2900355,Adustina (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2271.0,0.0,3.0,0.0,1990.0,0.0,0.0,83.0,0.0,443.0
2900405,Água Fria (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,265.0,0.0,360.0,4.0,6017.0,17.0,3.0,113.0,0.0,1342.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),383.0,0.0,6.0,0.0,0.0,55.0,0.0,0.0,88.0,0.0,...,16330.0,0.0,689.0,0.0,4539.0,13.0,22.0,1635.0,0.0,11885.0
2933406,Wagner (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,463.0,0.0,0.0,0.0,169.0,0.0,0.0,2.0,0.0,37.0
2933455,Wanderley (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4090.0,0.0,15.0,0.0,293.0,3.0,1.0,129.0,0.0,92.0


### Coelhos (valor da venda)

* Variável: Valor da venda de cabeças de coelhos nos estabelecimentos agropecuários

In [66]:
tabela='6934'
variavel='10052'

In [67]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                             })

In [68]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [69]:
subset_base.rename(columns={'Total': 'Coelho'}, inplace=True)

In [70]:
df_diversity=df_diversity.merge(subset_base, left_index=True, right_index=True, how='outer')
df_diversity

,variavel,Flores e folhagens para corte,Gramas,Mudas de plantas ornamentais,Mudas e outras formas de propagação (produzidas para plantio)_x,Plantas ornamentais em vaso,"Plantas, flores, folhagens medicinais",Sementes (produzidas para plantio)_x,Abacate,Abacaxi*,Alfafa fenada,...,Lã,Mel de abelha,Ovos de codorna,Ovos de galinha,Muares,Asininos,Equinos,Bubalinos,Galinhas,Coelho
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,92.0,0.0,0.0,0.0,0.0,27.0,0.0
2900207,Abaré (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,35.0,0.0,1.0,14.0,0.0,54.0,0.0
2900306,Acajutiba (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,20.0,0.0,21.0,0.0,0.0,0.0,0.0,35.0,0.0
2900355,Adustina (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,1990.0,0.0,0.0,83.0,0.0,443.0,0.0
2900405,Água Fria (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,360.0,4.0,6017.0,17.0,3.0,113.0,0.0,1342.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),383.0,0.0,6.0,0.0,0.0,55.0,0.0,0.0,88.0,0.0,...,0.0,689.0,0.0,4539.0,13.0,22.0,1635.0,0.0,11885.0,0.0
2933406,Wagner (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,0.0,0.0,0.0,169.0,0.0,0.0,2.0,0.0,37.0,0.0
2933455,Wanderley (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,15.0,0.0,293.0,3.0,1.0,129.0,0.0,92.0,0.0


### Codorna (valor da venda)

In [71]:
tabela='6933'
variavel='10048'

In [72]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                             })

In [73]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [74]:
subset_base.rename(columns={'Total': 'Codorna'}, inplace=True)

In [75]:
df_diversity=df_diversity.merge(subset_base, left_index=True, right_index=True, how='outer')
df_diversity

,variavel,Flores e folhagens para corte,Gramas,Mudas de plantas ornamentais,Mudas e outras formas de propagação (produzidas para plantio)_x,Plantas ornamentais em vaso,"Plantas, flores, folhagens medicinais",Sementes (produzidas para plantio)_x,Abacate,Abacaxi*,Alfafa fenada,...,Mel de abelha,Ovos de codorna,Ovos de galinha,Muares,Asininos,Equinos,Bubalinos,Galinhas,Coelho,Codorna
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,92.0,0.0,0.0,0.0,0.0,27.0,0.0,0.0
2900207,Abaré (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,35.0,0.0,1.0,14.0,0.0,54.0,0.0,0.0
2900306,Acajutiba (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,20.0,0.0,21.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0
2900355,Adustina (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,1990.0,0.0,0.0,83.0,0.0,443.0,0.0,7.0
2900405,Água Fria (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,360.0,4.0,6017.0,17.0,3.0,113.0,0.0,1342.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),383.0,0.0,6.0,0.0,0.0,55.0,0.0,0.0,88.0,0.0,...,689.0,0.0,4539.0,13.0,22.0,1635.0,0.0,11885.0,0.0,0.0
2933406,Wagner (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,0.0,0.0,169.0,0.0,0.0,2.0,0.0,37.0,0.0,0.0
2933455,Wanderley (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,15.0,0.0,293.0,3.0,1.0,129.0,0.0,92.0,0.0,0.0


### Ovino (valor da venda)

In [76]:
tabela='6930'
variavel='9759'

In [77]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                             })

In [78]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [79]:
subset_base.rename(columns={'Total': 'Ovino'}, inplace=True)

In [80]:
df_diversity=df_diversity.merge(subset_base, left_index=True, right_index=True, how='outer')
df_diversity

,variavel,Flores e folhagens para corte,Gramas,Mudas de plantas ornamentais,Mudas e outras formas de propagação (produzidas para plantio)_x,Plantas ornamentais em vaso,"Plantas, flores, folhagens medicinais",Sementes (produzidas para plantio)_x,Abacate,Abacaxi*,Alfafa fenada,...,Ovos de codorna,Ovos de galinha,Muares,Asininos,Equinos,Bubalinos,Galinhas,Coelho,Codorna,Ovino
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,92.0,0.0,0.0,0.0,0.0,27.0,0.0,0.0,9.0
2900207,Abaré (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,35.0,0.0,1.0,14.0,0.0,54.0,0.0,0.0,1194.0
2900306,Acajutiba (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,21.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0,27.0
2900355,Adustina (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1990.0,0.0,0.0,83.0,0.0,443.0,0.0,7.0,935.0
2900405,Água Fria (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,6017.0,17.0,3.0,113.0,0.0,1342.0,0.0,2.0,273.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),383.0,0.0,6.0,0.0,0.0,55.0,0.0,0.0,88.0,0.0,...,0.0,4539.0,13.0,22.0,1635.0,0.0,11885.0,0.0,0.0,431.0
2933406,Wagner (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,0.0,169.0,0.0,0.0,2.0,0.0,37.0,0.0,0.0,182.0
2933455,Wanderley (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,293.0,3.0,1.0,129.0,0.0,92.0,0.0,0.0,212.0


### Caprino (valor da venda)

In [81]:
tabela='6928'
variavel='9754'

In [82]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                             })

In [83]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [84]:
subset_base.rename(columns={'Total': 'Caprino'}, inplace=True)

In [85]:
df_diversity=df_diversity.merge(subset_base, left_index=True, right_index=True, how='outer')
df_diversity

,variavel,Flores e folhagens para corte,Gramas,Mudas de plantas ornamentais,Mudas e outras formas de propagação (produzidas para plantio)_x,Plantas ornamentais em vaso,"Plantas, flores, folhagens medicinais",Sementes (produzidas para plantio)_x,Abacate,Abacaxi*,Alfafa fenada,...,Ovos de galinha,Muares,Asininos,Equinos,Bubalinos,Galinhas,Coelho,Codorna,Ovino,Caprino
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,92.0,0.0,0.0,0.0,0.0,27.0,0.0,0.0,9.0,12.0
2900207,Abaré (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,35.0,0.0,1.0,14.0,0.0,54.0,0.0,0.0,1194.0,1164.0
2900306,Acajutiba (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,21.0,0.0,0.0,0.0,0.0,35.0,0.0,0.0,27.0,0.0
2900355,Adustina (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1990.0,0.0,0.0,83.0,0.0,443.0,0.0,7.0,935.0,70.0
2900405,Água Fria (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6017.0,17.0,3.0,113.0,0.0,1342.0,0.0,2.0,273.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),383.0,0.0,6.0,0.0,0.0,55.0,0.0,0.0,88.0,0.0,...,4539.0,13.0,22.0,1635.0,0.0,11885.0,0.0,0.0,431.0,294.0
2933406,Wagner (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,169.0,0.0,0.0,2.0,0.0,37.0,0.0,0.0,182.0,6.0
2933455,Wanderley (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,293.0,3.0,1.0,129.0,0.0,92.0,0.0,0.0,212.0,59.0


### Suinos (valor da venda)

In [86]:
tabela='6926'
variavel='9769'

In [87]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                             })

In [88]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [89]:
subset_base.rename(columns={'Total': 'Suíno'}, inplace=True)

In [90]:
df_diversity=df_diversity.merge(subset_base, left_index=True, right_index=True, how='outer')
df_diversity

,variavel,Flores e folhagens para corte,Gramas,Mudas de plantas ornamentais,Mudas e outras formas de propagação (produzidas para plantio)_x,Plantas ornamentais em vaso,"Plantas, flores, folhagens medicinais",Sementes (produzidas para plantio)_x,Abacate,Abacaxi*,Alfafa fenada,...,Muares,Asininos,Equinos,Bubalinos,Galinhas,Coelho,Codorna,Ovino,Caprino,Suíno
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,27.0,0.0,0.0,9.0,12.0,99.0
2900207,Abaré (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,14.0,0.0,54.0,0.0,0.0,1194.0,1164.0,116.0
2900306,Acajutiba (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,35.0,0.0,0.0,27.0,0.0,215.0
2900355,Adustina (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,83.0,0.0,443.0,0.0,7.0,935.0,70.0,278.0
2900405,Água Fria (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,17.0,3.0,113.0,0.0,1342.0,0.0,2.0,273.0,22.0,267.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),383.0,0.0,6.0,0.0,0.0,55.0,0.0,0.0,88.0,0.0,...,13.0,22.0,1635.0,0.0,11885.0,0.0,0.0,431.0,294.0,11417.0
2933406,Wagner (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,0.0,0.0,2.0,0.0,37.0,0.0,0.0,182.0,6.0,53.0
2933455,Wanderley (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,1.0,129.0,0.0,92.0,0.0,0.0,212.0,59.0,990.0


### Rã (valor da venda)

In [91]:
tabela='6939'
variavel='10065'

In [92]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                             })

In [93]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [94]:
subset_base.rename(columns={'Total': 'Rã'}, inplace=True)

In [95]:
df_diversity=df_diversity.merge(subset_base, left_index=True, right_index=True, how='outer')
df_diversity

,variavel,Flores e folhagens para corte,Gramas,Mudas de plantas ornamentais,Mudas e outras formas de propagação (produzidas para plantio)_x,Plantas ornamentais em vaso,"Plantas, flores, folhagens medicinais",Sementes (produzidas para plantio)_x,Abacate,Abacaxi*,Alfafa fenada,...,Asininos,Equinos,Bubalinos,Galinhas,Coelho,Codorna,Ovino,Caprino,Suíno,Rã
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,27.0,0.0,0.0,9.0,12.0,99.0,0
2900207,Abaré (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,14.0,0.0,54.0,0.0,0.0,1194.0,1164.0,116.0,0
2900306,Acajutiba (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,35.0,0.0,0.0,27.0,0.0,215.0,0
2900355,Adustina (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,83.0,0.0,443.0,0.0,7.0,935.0,70.0,278.0,0
2900405,Água Fria (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,113.0,0.0,1342.0,0.0,2.0,273.0,22.0,267.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),383.0,0.0,6.0,0.0,0.0,55.0,0.0,0.0,88.0,0.0,...,22.0,1635.0,0.0,11885.0,0.0,0.0,431.0,294.0,11417.0,0
2933406,Wagner (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,0.0,2.0,0.0,37.0,0.0,0.0,182.0,6.0,53.0,0
2933455,Wanderley (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,129.0,0.0,92.0,0.0,0.0,212.0,59.0,990.0,0


### Bovino

In [96]:
df_bovino=pd.DataFrame()

In [97]:
tabela='6910'
variavel='9743' # Valor da venda de cabeças de bovinos nos estabelecimentos agropecuários com 50 cabeças e menos

In [98]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                             })

In [99]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [100]:
subset_base.rename(columns={'Total': 'Bovino_menos50'}, inplace=True)

In [101]:
df_bovino=subset_base

In [102]:
tabela='6910'
variavel='9749' #Valor da venda de cabeças de bovinos para abate nos estabelecimentos agropecuários com mais de 50 cabeças

In [103]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                             })

In [104]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [105]:
subset_base.rename(columns={'Total': 'Bovino_abate'}, inplace=True)

In [106]:
df_bovino=df_bovino.merge(subset_base, left_index=True, right_index=True, how='outer')

In [107]:
tabela='6910'
variavel='9747' #Valor da venda de cabeças de bovinos para cria, recria ou engorda nos estabelecimentos agropecuários com mais de 50 cabeças

In [108]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                             })

In [109]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [110]:
subset_base.rename(columns={'Total': 'Bovino_cria'}, inplace=True)

In [111]:
df_bovino=df_bovino.merge(subset_base, left_index=True, right_index=True, how='outer')

In [112]:
df_bovino['bovino']=df_bovino['Bovino_menos50']+df_bovino['Bovino_abate']+df_bovino['Bovino_cria']

In [113]:
df_diversity=df_diversity.merge(df_bovino[['bovino']], left_index=True, right_index=True, how='outer')
df_diversity

,variavel,Flores e folhagens para corte,Gramas,Mudas de plantas ornamentais,Mudas e outras formas de propagação (produzidas para plantio)_x,Plantas ornamentais em vaso,"Plantas, flores, folhagens medicinais",Sementes (produzidas para plantio)_x,Abacate,Abacaxi*,Alfafa fenada,...,Equinos,Bubalinos,Galinhas,Coelho,Codorna,Ovino,Caprino,Suíno,Rã,bovino
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,27.0,0.0,0.0,9.0,12.0,99.0,0,1350.0
2900207,Abaré (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.0,0.0,54.0,0.0,0.0,1194.0,1164.0,116.0,0,1003.0
2900306,Acajutiba (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,35.0,0.0,0.0,27.0,0.0,215.0,0,2797.0
2900355,Adustina (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,83.0,0.0,443.0,0.0,7.0,935.0,70.0,278.0,0,3997.0
2900405,Água Fria (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,113.0,0.0,1342.0,0.0,2.0,273.0,22.0,267.0,0,9367.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),383.0,0.0,6.0,0.0,0.0,55.0,0.0,0.0,88.0,0.0,...,1635.0,0.0,11885.0,0.0,0.0,431.0,294.0,11417.0,0,18969.0
2933406,Wagner (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,2.0,0.0,37.0,0.0,0.0,182.0,6.0,53.0,0,544.0
2933455,Wanderley (BA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,129.0,0.0,92.0,0.0,0.0,212.0,59.0,990.0,0,20559.0


## Índice

In [114]:
df_diversity['total']=0.0
for col in df_diversity.columns:
    df_diversity['total']+=df_diversity[col]

In [115]:
df_diversity['indice']=0.0
for col in df_diversity.columns:
    if col!='total':
        df_diversity['indice']+=((df_diversity[col]/df_diversity['total'])**2)

In [116]:
df_diversity['indice']=1.0-df_diversity['indice']

In [117]:
df_diversity[['indice']]

,variavel,indice
cod_municipio,nome_municipio,
2900108,Abaíra (BA),0.936468
2900207,Abaré (BA),0.921359
2900306,Acajutiba (BA),0.937661
2900355,Adustina (BA),0.914826
2900405,Água Fria (BA),0.925353
...,...,...
2933307,Vitória da Conquista (BA),0.970819
2933406,Wagner (BA),0.898382
2933455,Wanderley (BA),0.908981


In [118]:
df_diversity.to_csv('saidas/indice_diversidade.csv')

# Variáveis pra análise fatorial

## Dimensão educação 

### Acesso à escola
* Variável: número de estabelecimetos agropecuários dirigidos pelo produtor
    * Classificação Escolaridade do produtor 800
        * **Nunca frequentou a escola** - 40708
        * Total (para extrair percentual) - 41147

In [119]:
tabela='6755'
variavel='9998'
classificacao='800'
classes='[40708,41147]'

In [120]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                              'classificacao': classificacao+classes
                             })

In [121]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [122]:
for col in set(subset_base.columns)-set(['Total']):
    subset_base['Acesso à escola']=subset_base[col]/subset_base['Total']*100

In [123]:
base_final=subset_base[['Acesso à escola']]

### Alfabetização na idade devida

* Variável: número de estabelecimetos agropecuários dirigidos pelo produtor
    * Classificação Escolaridade do produtor 800
        * **Alfabetização para jovens e adultos** (AJA) - 40710
        * **Não sabe ler e escrever** - 46511
        * Total (para extrair percentual) - 41147

In [124]:
tabela='6755'
variavel='9998'
classificacao='800'
classes='[40710,46511,41147]'

In [125]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                              'classificacao': classificacao+classes
                             })

In [126]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [127]:
for col in set(subset_base.columns)-set(['Total']):
    subset_base[col+' %']=subset_base[col]/subset_base['Total']*100

subset_base['Alfabetização na idade devida']= subset_base['Sabe ler e escrever - não %']+subset_base['Alfabetização de jovens e adultos - AJA %']

In [128]:
subset_base

,variavel,Alfabetização de jovens e adultos - AJA,Sabe ler e escrever - não,Total,Sabe ler e escrever - não %,Alfabetização de jovens e adultos - AJA %,Alfabetização na idade devida
cod_municipio,nome_municipio,,,,,,
2900108,Abaíra (BA),5.0,141.0,919.0,15.342764,0.544070,15.886834
2900207,Abaré (BA),47.0,506.0,1883.0,26.872013,2.496017,29.368030
2900306,Acajutiba (BA),12.0,377.0,1335.0,28.239700,0.898876,29.138577
2900355,Adustina (BA),35.0,1231.0,3249.0,37.888581,1.077255,38.965836
2900405,Água Fria (BA),101.0,692.0,2159.0,32.051876,4.678092,36.729968
...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),125.0,2621.0,6538.0,40.088712,1.911900,42.000612
2933406,Wagner (BA),10.0,195.0,721.0,27.045770,1.386963,28.432732
2933455,Wanderley (BA),22.0,380.0,1659.0,22.905365,1.326100,24.231465


In [129]:
base_final=base_final.merge(subset_base[['Alfabetização na idade devida']], left_index=True, right_index=True, how='outer')
base_final

,variavel,Acesso à escola,Alfabetização na idade devida
cod_municipio,nome_municipio,,
2900108,Abaíra (BA),12.078346,15.886834
2900207,Abaré (BA),8.125332,29.368030
2900306,Acajutiba (BA),21.273408,29.138577
2900355,Adustina (BA),22.930132,38.965836
2900405,Água Fria (BA),17.924965,36.729968
...,...,...,...
2933307,Vitória da Conquista (BA),33.221169,42.000612
2933406,Wagner (BA),26.490985,28.432732
2933455,Wanderley (BA),18.866787,24.231465


### Orientação técnica

* Variável: número de estabelecimetos agropecuários 183
    * Classificação Origem da orientação técnica recebida 12567
        * **Não recebe** - 113559
        * Total (para extrair percentual) - 41151

In [130]:
tabela='6906'
variavel='183'
classificacao='12567'
classes='[113559,41151]'

In [131]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                              'classificacao': classificacao+classes
                             })

In [132]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [133]:
for col in set(subset_base.columns)-set(['Total']):
    subset_base[col+' %']=subset_base[col]/subset_base['Total']*100

In [134]:
subset_base

,variavel,Não recebe,Total,Não recebe %
cod_municipio,nome_municipio,,,
2900108,Abaíra (BA),887.0,919.0,96.517954
2900207,Abaré (BA),1790.0,1885.0,94.960212
2900306,Acajutiba (BA),1268.0,1337.0,94.839192
2900355,Adustina (BA),2552.0,3249.0,78.547245
2900405,Água Fria (BA),1945.0,2165.0,89.838337
...,...,...,...,...
2933307,Vitória da Conquista (BA),6078.0,6548.0,92.822236
2933406,Wagner (BA),640.0,724.0,88.397790
2933455,Wanderley (BA),1554.0,1660.0,93.614458


In [135]:
base_final=base_final.merge(subset_base[['Não recebe %']], left_index=True, right_index=True, how='outer')
base_final.rename(columns={'Não recebe %': 'Orientação técnica'}, inplace=True)

In [136]:
base_final

,variavel,Acesso à escola,Alfabetização na idade devida,Orientação técnica
cod_municipio,nome_municipio,,,
2900108,Abaíra (BA),12.078346,15.886834,96.517954
2900207,Abaré (BA),8.125332,29.368030,94.960212
2900306,Acajutiba (BA),21.273408,29.138577,94.839192
2900355,Adustina (BA),22.930132,38.965836,78.547245
2900405,Água Fria (BA),17.924965,36.729968,89.838337
...,...,...,...,...
2933307,Vitória da Conquista (BA),33.221169,42.000612,92.822236
2933406,Wagner (BA),26.490985,28.432732,88.397790
2933455,Wanderley (BA),18.866787,24.231465,93.614458


### Demais informações técnicas


* Variável: número de estabelecimetos agropecuários 183
    * Forma de obtenção de informações técnicas recebidas pelo estabelecimento 837
        * **Não obtém informações técnicas** - 46543
        * Total (para extrair percentual) - 46544

In [137]:
tabela='6781'
variavel='183'
classificacao='837'
classes='[46543,46544]'

In [138]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                              'classificacao': classificacao+classes
                             })

In [139]:
subset_base=process_json_censo_agro(response.json(), classificacao)

for col in set(subset_base.columns)-set(['Total']):
    subset_base[col+' %']=subset_base[col]/subset_base['Total']*100

In [140]:
subset_base

,variavel,Não obtém informações técnicas,Total,Não obtém informações técnicas %
cod_municipio,nome_municipio,,,
2900108,Abaíra (BA),249.0,919.0,27.094668
2900207,Abaré (BA),888.0,1885.0,47.108753
2900306,Acajutiba (BA),786.0,1337.0,58.788332
2900355,Adustina (BA),1443.0,3249.0,44.413666
2900405,Água Fria (BA),1198.0,2165.0,55.334873
...,...,...,...,...
2933307,Vitória da Conquista (BA),938.0,6548.0,14.324985
2933406,Wagner (BA),440.0,724.0,60.773481
2933455,Wanderley (BA),135.0,1660.0,8.132530


In [141]:
base_final=base_final.merge(subset_base[['Não obtém informações técnicas %']], left_index=True, right_index=True, how='outer')
base_final.rename(columns={'Não obtém informações técnicas %': 'Demais informações técnicas'}, inplace=True)

In [142]:
base_final

,variavel,Acesso à escola,Alfabetização na idade devida,Orientação técnica,Demais informações técnicas
cod_municipio,nome_municipio,,,,
2900108,Abaíra (BA),12.078346,15.886834,96.517954,27.094668
2900207,Abaré (BA),8.125332,29.368030,94.960212,47.108753
2900306,Acajutiba (BA),21.273408,29.138577,94.839192,58.788332
2900355,Adustina (BA),22.930132,38.965836,78.547245,44.413666
2900405,Água Fria (BA),17.924965,36.729968,89.838337,55.334873
...,...,...,...,...,...
2933307,Vitória da Conquista (BA),33.221169,42.000612,92.822236,14.324985
2933406,Wagner (BA),26.490985,28.432732,88.397790,60.773481
2933455,Wanderley (BA),18.866787,24.231465,93.614458,8.132530


## Estruturas sociais e institucionais 

### Participação em grupos da sociedade civil

* Variável: número de estabelecimetos agropecuários 183
    * Associação do produtor à cooperativa e/ou à entidade de classe 12598
        * **Não é associado** - 113457
        * Total (para extrair percentual) - 41141

In [143]:
tabela='6846'
variavel='183'
classificacao='12598'
classes='[113457,41141]'

In [144]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                              'classificacao': classificacao+classes
                             })

In [145]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [146]:
for col in set(subset_base.columns)-set(['Total']):
    subset_base[col+' %']=subset_base[col]/subset_base['Total']*100

In [147]:
subset_base

,variavel,Não é associado,Total,Não é associado %
cod_municipio,nome_municipio,,,
2900108,Abaíra (BA),703.0,919.0,76.496192
2900207,Abaré (BA),1458.0,1885.0,77.347480
2900306,Acajutiba (BA),942.0,1337.0,70.456245
2900355,Adustina (BA),2833.0,3249.0,87.196060
2900405,Água Fria (BA),1183.0,2165.0,54.642032
...,...,...,...,...
2933307,Vitória da Conquista (BA),4233.0,6548.0,64.645693
2933406,Wagner (BA),536.0,724.0,74.033149
2933455,Wanderley (BA),1058.0,1660.0,63.734940


In [148]:
base_final=base_final.merge(subset_base[['Não é associado %']], left_index=True, right_index=True, how='outer')
base_final.rename(columns={'Não é associado %': 'Participação em grupos da sociedade civil'}, inplace=True)

In [149]:
base_final

,variavel,Acesso à escola,Alfabetização na idade devida,Orientação técnica,Demais informações técnicas,Participação em grupos da sociedade civil
cod_municipio,nome_municipio,,,,,
2900108,Abaíra (BA),12.078346,15.886834,96.517954,27.094668,76.496192
2900207,Abaré (BA),8.125332,29.368030,94.960212,47.108753,77.347480
2900306,Acajutiba (BA),21.273408,29.138577,94.839192,58.788332,70.456245
2900355,Adustina (BA),22.930132,38.965836,78.547245,44.413666,87.196060
2900405,Água Fria (BA),17.924965,36.729968,89.838337,55.334873,54.642032
...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),33.221169,42.000612,92.822236,14.324985,64.645693
2933406,Wagner (BA),26.490985,28.432732,88.397790,60.773481,74.033149
2933455,Wanderley (BA),18.866787,24.231465,93.614458,8.132530,63.734940


### Acesso a crédito e financiamento

* Variável: Número de estabelecimentos agropecuários que obtiveram financiamento - 1990 
    * obs: não encontrei o código da variável em percentual: percentual do total geral (%): 5 de 5 casas decimais -- no entanto, analisando os resultados da tabela, o percentual é sobre o total que tiveram financiamento, nao sobre o total de estabelecimentos.
    
Portanto, vamos lidar com 2 tabelas e 2 variáveis:
* Número de estabelecimentos agropecuários que obtiveram financiamento - 1990 - da tabela 6895
* Número de estabelecimentos agropecuários - 183 - da tabela 6846 (usada anteriormente)

**Tabela1 - financiamento**

In [153]:
tabela='6895'
variavel='1990'

In [154]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba
                             })

In [155]:
subset_base_1=process_json_censo_agro(response.json(), '')

**Tabela2 - N de estabelecimentos**

In [156]:
tabela='6846'
variavel='183'

In [157]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba
                             })

In [158]:
subset_base=process_json_censo_agro(response.json(), None)

In [159]:
subset_base

,variavel,Total
cod_municipio,nome_municipio,
2900108,Abaíra (BA),919.0
2900207,Abaré (BA),1885.0
2900306,Acajutiba (BA),1337.0
2900355,Adustina (BA),3249.0
2900405,Água Fria (BA),2165.0
...,...,...
2933307,Vitória da Conquista (BA),6548.0
2933406,Wagner (BA),724.0
2933455,Wanderley (BA),1660.0


In [160]:
subset_base_1.rename(columns={'Total': 'Financiada'}, inplace=True)
subset_base_1

,variavel,Financiada
cod_municipio,nome_municipio,
2900108,Abaíra (BA),57.0
2900207,Abaré (BA),221.0
2900306,Acajutiba (BA),43.0
2900355,Adustina (BA),556.0
2900405,Água Fria (BA),259.0
...,...,...
2933307,Vitória da Conquista (BA),698.0
2933406,Wagner (BA),24.0
2933455,Wanderley (BA),282.0


In [161]:
subset_base=subset_base.merge(subset_base_1, left_index=True, right_index=True, how='outer')

In [162]:
subset_base['Não financiada']=subset_base['Total']-subset_base['Financiada']
subset_base['Não financiada %']=subset_base['Não financiada']/subset_base['Total']*100

In [163]:
subset_base

,variavel,Total,Financiada,Não financiada,Não financiada %
cod_municipio,nome_municipio,,,,
2900108,Abaíra (BA),919.0,57.0,862.0,93.797606
2900207,Abaré (BA),1885.0,221.0,1664.0,88.275862
2900306,Acajutiba (BA),1337.0,43.0,1294.0,96.783844
2900355,Adustina (BA),3249.0,556.0,2693.0,82.887042
2900405,Água Fria (BA),2165.0,259.0,1906.0,88.036952
...,...,...,...,...,...
2933307,Vitória da Conquista (BA),6548.0,698.0,5850.0,89.340257
2933406,Wagner (BA),724.0,24.0,700.0,96.685083
2933455,Wanderley (BA),1660.0,282.0,1378.0,83.012048


In [164]:
base_final=base_final.merge(subset_base[['Não financiada %']], left_index=True, right_index=True, how='outer')
base_final.rename(columns={'Não financiada %': 'Acesso a crédito e financiamento'}, inplace=True)

In [165]:
base_final

,variavel,Acesso à escola,Alfabetização na idade devida,Orientação técnica,Demais informações técnicas,Participação em grupos da sociedade civil,Acesso a crédito e financiamento
cod_municipio,nome_municipio,,,,,,
2900108,Abaíra (BA),12.078346,15.886834,96.517954,27.094668,76.496192,93.797606
2900207,Abaré (BA),8.125332,29.368030,94.960212,47.108753,77.347480,88.275862
2900306,Acajutiba (BA),21.273408,29.138577,94.839192,58.788332,70.456245,96.783844
2900355,Adustina (BA),22.930132,38.965836,78.547245,44.413666,87.196060,82.887042
2900405,Água Fria (BA),17.924965,36.729968,89.838337,55.334873,54.642032,88.036952
...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),33.221169,42.000612,92.822236,14.324985,64.645693,89.340257
2933406,Wagner (BA),26.490985,28.432732,88.397790,60.773481,74.033149,96.685083
2933455,Wanderley (BA),18.866787,24.231465,93.614458,8.132530,63.734940,83.012048


### Acesso a benefícios sociais -- 4 variáveis (que nao recebeu)

* Variável: - Número de estabelecimentos agropecuários que obtiveram receitas ou com outras rendas do produtor 10006
    * 841- Receitas do estabelecimento e outras rendas do produtor
        * '46601': 'Outras receitas do produtor - recursos de aposentadorias ou pensões'
        * '46605': 'Outras receitas do produtor - recebimento do Programa Nacional de Habitação Rural Minha Casa Minha Vida'
        * '46606': 'Outras receitas do produtor - recebimento de pagamento por serviços ambientais (Bolsa Verde e Programas Estaduais)'
        * '46607': 'Outras receitas do produtor - provenientes de programas dos Governos (federal, estadual ou municipal)'

In [166]:
tabela='6901'
variavel='10006'
classificacao='841'
classes='[46601,46605,46606,46607]'

In [167]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                              'classificacao': classificacao+classes
                             })

In [168]:
subset_base=process_json_censo_agro(response.json(), classificacao)

* pegando total da variável: número de estabelecimentos agropecuários 183

In [169]:
tabela='6846'
variavel='183'

In [170]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba
                             })

In [171]:
total=process_json_censo_agro(response.json(), None)

In [172]:
for col in subset_base.columns:
    subset_base[col+' %']=(total['Total']-subset_base[col])/total['Total']*100
    subset_base.drop(col, axis=1, inplace=True)

In [173]:
subset_base

,variavel,"Outras receitas do produtor - provenientes de programas dos Governos (federal, estadual ou municipal) %",Outras receitas do produtor - recebimento de pagamento por serviços ambientais (Bolsa Verde e Programas Estaduais) %,Outras receitas do produtor - recebimento do Programa Nacional de Habitação Rural Minha Casa Minha Vida %,Outras receitas do produtor - recursos de aposentadorias ou pensões %
cod_municipio,nome_municipio,,,,
2900108,Abaíra (BA),69.205658,99.782372,99.455930,52.013058
2900207,Abaré (BA),61.591512,99.946950,99.098143,69.071618
2900306,Acajutiba (BA),94.016455,100.000000,99.925206,70.680628
2900355,Adustina (BA),76.546630,100.000000,100.000000,67.405355
2900405,Água Fria (BA),66.189376,100.000000,99.769053,56.212471
...,...,...,...,...,...
2933307,Vitória da Conquista (BA),81.795968,99.893097,99.679291,53.726329
2933406,Wagner (BA),84.806630,100.000000,100.000000,72.790055
2933455,Wanderley (BA),66.807229,99.397590,100.000000,55.963855


In [174]:
base_final=base_final.merge(subset_base, left_index=True, right_index=True, how='outer')
base_final.rename(columns={
    'Outras receitas do produtor - provenientes de programas dos Governos (federal, estadual ou municipal) %': 
    'Acesso a outros programas do governo',
    'Outras receitas do produtor - recebimento de pagamento por serviços ambientais (Bolsa Verde e Programas Estaduais) %': 
    'Participação nos programas de pagamento por serviços ambientais',
    'Outras receitas do produtor - recebimento do Programa Nacional de Habitação Rural Minha Casa Minha Vida %': 
    'Participação no Programa Nacional de Habitação Rural',
    'Outras receitas do produtor - recursos de aposentadorias ou pensões %': 'Acesso a aposentadoria ou pensão'
}, inplace=True)

In [175]:
base_final

,variavel,Acesso à escola,Alfabetização na idade devida,Orientação técnica,Demais informações técnicas,Participação em grupos da sociedade civil,Acesso a crédito e financiamento,Acesso a outros programas do governo,Participação nos programas de pagamento por serviços ambientais,Participação no Programa Nacional de Habitação Rural,Acesso a aposentadoria ou pensão
cod_municipio,nome_municipio,,,,,,,,,,
2900108,Abaíra (BA),12.078346,15.886834,96.517954,27.094668,76.496192,93.797606,69.205658,99.782372,99.455930,52.013058
2900207,Abaré (BA),8.125332,29.368030,94.960212,47.108753,77.347480,88.275862,61.591512,99.946950,99.098143,69.071618
2900306,Acajutiba (BA),21.273408,29.138577,94.839192,58.788332,70.456245,96.783844,94.016455,100.000000,99.925206,70.680628
2900355,Adustina (BA),22.930132,38.965836,78.547245,44.413666,87.196060,82.887042,76.546630,100.000000,100.000000,67.405355
2900405,Água Fria (BA),17.924965,36.729968,89.838337,55.334873,54.642032,88.036952,66.189376,100.000000,99.769053,56.212471
...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),33.221169,42.000612,92.822236,14.324985,64.645693,89.340257,81.795968,99.893097,99.679291,53.726329
2933406,Wagner (BA),26.490985,28.432732,88.397790,60.773481,74.033149,96.685083,84.806630,100.000000,100.000000,72.790055
2933455,Wanderley (BA),18.866787,24.231465,93.614458,8.132530,63.734940,83.012048,66.807229,99.397590,100.000000,55.963855


## Padrão de vida

### Condição legal das terras

* Variável: número de estabelecimetos agropecuários 183
    * Condição legal das terras - 221
    * Todas (111195), exceto classe própria (111134)

In [176]:
tabela='6753'
variavel='183'
classificacao='221'
classes='[111195,111134]'

In [177]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                              'classificacao': classificacao+classes
                             })

In [178]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [179]:
subset_base

,variavel,Próprias,Total
cod_municipio,nome_municipio,,
2900108,Abaíra (BA),862.0,919.0
2900207,Abaré (BA),1348.0,1885.0
2900306,Acajutiba (BA),1301.0,1337.0
2900355,Adustina (BA),2842.0,3249.0
2900405,Água Fria (BA),1795.0,2165.0
...,...,...,...
2933307,Vitória da Conquista (BA),5174.0,6548.0
2933406,Wagner (BA),716.0,724.0
2933455,Wanderley (BA),1653.0,1660.0


In [180]:
subset_base['Não próprias']=subset_base['Total']-subset_base['Próprias']
subset_base['Não próprias %']=subset_base['Não próprias']/subset_base['Total']*100

In [181]:
base_final=base_final.merge(subset_base[['Não próprias %']], left_index=True, right_index=True, how='outer')
base_final.rename(columns={'Não próprias %': 'Condição legal das terras'}, inplace=True)

In [182]:
base_final

,variavel,Acesso à escola,Alfabetização na idade devida,Orientação técnica,Demais informações técnicas,Participação em grupos da sociedade civil,Acesso a crédito e financiamento,Acesso a outros programas do governo,Participação nos programas de pagamento por serviços ambientais,Participação no Programa Nacional de Habitação Rural,Acesso a aposentadoria ou pensão,Condição legal das terras
cod_municipio,nome_municipio,,,,,,,,,,,
2900108,Abaíra (BA),12.078346,15.886834,96.517954,27.094668,76.496192,93.797606,69.205658,99.782372,99.455930,52.013058,6.202394
2900207,Abaré (BA),8.125332,29.368030,94.960212,47.108753,77.347480,88.275862,61.591512,99.946950,99.098143,69.071618,28.488064
2900306,Acajutiba (BA),21.273408,29.138577,94.839192,58.788332,70.456245,96.783844,94.016455,100.000000,99.925206,70.680628,2.692595
2900355,Adustina (BA),22.930132,38.965836,78.547245,44.413666,87.196060,82.887042,76.546630,100.000000,100.000000,67.405355,12.526931
2900405,Água Fria (BA),17.924965,36.729968,89.838337,55.334873,54.642032,88.036952,66.189376,100.000000,99.769053,56.212471,17.090069
...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),33.221169,42.000612,92.822236,14.324985,64.645693,89.340257,81.795968,99.893097,99.679291,53.726329,20.983506
2933406,Wagner (BA),26.490985,28.432732,88.397790,60.773481,74.033149,96.685083,84.806630,100.000000,100.000000,72.790055,1.104972
2933455,Wanderley (BA),18.866787,24.231465,93.614458,8.132530,63.734940,83.012048,66.807229,99.397590,100.000000,55.963855,0.421687


### Eletricidade no domicílio

* Variável: número de estabelecimetos agropecuários 183
    * Existência de energia elétrica - 309
        * **Não tinham** - 3018
        * Total (para extrair percentual) - 10969

In [183]:
tabela='6778'
variavel='183'
classificacao='309'
classes='[3018,10969]'

In [184]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                              'classificacao': classificacao+classes
                             })

In [185]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [186]:
for col in set(subset_base.columns)-set(['Total']):
    subset_base[col+' %']=subset_base[col]/subset_base['Total']*100

In [187]:
subset_base

,variavel,Não tinham,Total,Não tinham %
cod_municipio,nome_municipio,,,
2900108,Abaíra (BA),255.0,919.0,27.747552
2900207,Abaré (BA),1095.0,1885.0,58.090186
2900306,Acajutiba (BA),621.0,1337.0,46.447270
2900355,Adustina (BA),1526.0,3249.0,46.968298
2900405,Água Fria (BA),504.0,2165.0,23.279446
...,...,...,...,...
2933307,Vitória da Conquista (BA),668.0,6548.0,10.201588
2933406,Wagner (BA),187.0,724.0,25.828729
2933455,Wanderley (BA),290.0,1660.0,17.469880


In [188]:
base_final=base_final.merge(subset_base[['Não tinham %']], left_index=True, right_index=True, how='outer')
base_final.rename(columns={'Não tinham %': 'Eletricidade no domicílio'}, inplace=True)

In [189]:
base_final

,variavel,Acesso à escola,Alfabetização na idade devida,Orientação técnica,Demais informações técnicas,Participação em grupos da sociedade civil,Acesso a crédito e financiamento,Acesso a outros programas do governo,Participação nos programas de pagamento por serviços ambientais,Participação no Programa Nacional de Habitação Rural,Acesso a aposentadoria ou pensão,Condição legal das terras,Eletricidade no domicílio
cod_municipio,nome_municipio,,,,,,,,,,,,
2900108,Abaíra (BA),12.078346,15.886834,96.517954,27.094668,76.496192,93.797606,69.205658,99.782372,99.455930,52.013058,6.202394,27.747552
2900207,Abaré (BA),8.125332,29.368030,94.960212,47.108753,77.347480,88.275862,61.591512,99.946950,99.098143,69.071618,28.488064,58.090186
2900306,Acajutiba (BA),21.273408,29.138577,94.839192,58.788332,70.456245,96.783844,94.016455,100.000000,99.925206,70.680628,2.692595,46.447270
2900355,Adustina (BA),22.930132,38.965836,78.547245,44.413666,87.196060,82.887042,76.546630,100.000000,100.000000,67.405355,12.526931,46.968298
2900405,Água Fria (BA),17.924965,36.729968,89.838337,55.334873,54.642032,88.036952,66.189376,100.000000,99.769053,56.212471,17.090069,23.279446
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),33.221169,42.000612,92.822236,14.324985,64.645693,89.340257,81.795968,99.893097,99.679291,53.726329,20.983506,10.201588
2933406,Wagner (BA),26.490985,28.432732,88.397790,60.773481,74.033149,96.685083,84.806630,100.000000,100.000000,72.790055,1.104972,25.828729
2933455,Wanderley (BA),18.866787,24.231465,93.614458,8.132530,63.734940,83.012048,66.807229,99.397590,100.000000,55.963855,0.421687,17.469880


### Comunicação via telefone

* Variável: número de estabelecimetos agropecuários 183
    * Telefone, email e internet - 798
        * **40693**: 'Existência de telefone - não'
        * 47179 - Total (para extrair percentual) 

In [190]:
tabela='6962'
variavel='183'
classificacao='798'
classes='[40693,47179]'

In [191]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                              'classificacao': classificacao+classes
                             })

In [192]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [193]:
for col in set(subset_base.columns)-set(['Total']):
    subset_base[col+' %']=subset_base[col]/subset_base['Total']*100

In [194]:
base_final=base_final.merge(subset_base[['Existência de telefone - não %']], left_index=True, right_index=True, how='outer')
base_final.rename(columns={'Existência de telefone - não %': 'Comunicação via telefone'}, inplace=True)

In [195]:
base_final

,variavel,Acesso à escola,Alfabetização na idade devida,Orientação técnica,Demais informações técnicas,Participação em grupos da sociedade civil,Acesso a crédito e financiamento,Acesso a outros programas do governo,Participação nos programas de pagamento por serviços ambientais,Participação no Programa Nacional de Habitação Rural,Acesso a aposentadoria ou pensão,Condição legal das terras,Eletricidade no domicílio,Comunicação via telefone
cod_municipio,nome_municipio,,,,,,,,,,,,,
2900108,Abaíra (BA),12.078346,15.886834,96.517954,27.094668,76.496192,93.797606,69.205658,99.782372,99.455930,52.013058,6.202394,27.747552,46.463547
2900207,Abaré (BA),8.125332,29.368030,94.960212,47.108753,77.347480,88.275862,61.591512,99.946950,99.098143,69.071618,28.488064,58.090186,56.498674
2900306,Acajutiba (BA),21.273408,29.138577,94.839192,58.788332,70.456245,96.783844,94.016455,100.000000,99.925206,70.680628,2.692595,46.447270,60.433807
2900355,Adustina (BA),22.930132,38.965836,78.547245,44.413666,87.196060,82.887042,76.546630,100.000000,100.000000,67.405355,12.526931,46.968298,47.122191
2900405,Água Fria (BA),17.924965,36.729968,89.838337,55.334873,54.642032,88.036952,66.189376,100.000000,99.769053,56.212471,17.090069,23.279446,29.699769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),33.221169,42.000612,92.822236,14.324985,64.645693,89.340257,81.795968,99.893097,99.679291,53.726329,20.983506,10.201588,50.717776
2933406,Wagner (BA),26.490985,28.432732,88.397790,60.773481,74.033149,96.685083,84.806630,100.000000,100.000000,72.790055,1.104972,25.828729,56.491713
2933455,Wanderley (BA),18.866787,24.231465,93.614458,8.132530,63.734940,83.012048,66.807229,99.397590,100.000000,55.963855,0.421687,17.469880,35.963855


### Acesso à internet

* Variável: número de estabelecimetos agropecuários 183
    * Telefone, email e internet - 798
        * **40697**: 'Acesso à internet - não'
        * 47179 - Total (para extrair percentual) 

In [196]:
tabela='6962'
variavel='183'
classificacao='798'
classes='[40697,47179]'

In [197]:
response=requests.get(url_base+'/agregados/'+tabela+'/periodos/2017/variaveis/'+variavel,      
                      params={'localidades': localidade+cod_cidades_ba,
                              'classificacao': classificacao+classes
                             })

In [198]:
subset_base=process_json_censo_agro(response.json(), classificacao)

In [199]:
for col in set(subset_base.columns)-set(['Total']):
    subset_base[col+' %']=subset_base[col]/subset_base['Total']*100

In [200]:
base_final=base_final.merge(subset_base[['Acesso à internet - não %']], left_index=True, right_index=True, how='outer')
base_final.rename(columns={'Acesso à internet - não %': 'Acesso à internet'}, inplace=True)

In [201]:
base_final

,variavel,Acesso à escola,Alfabetização na idade devida,Orientação técnica,Demais informações técnicas,Participação em grupos da sociedade civil,Acesso a crédito e financiamento,Acesso a outros programas do governo,Participação nos programas de pagamento por serviços ambientais,Participação no Programa Nacional de Habitação Rural,Acesso a aposentadoria ou pensão,Condição legal das terras,Eletricidade no domicílio,Comunicação via telefone,Acesso à internet
cod_municipio,nome_municipio,,,,,,,,,,,,,,
2900108,Abaíra (BA),12.078346,15.886834,96.517954,27.094668,76.496192,93.797606,69.205658,99.782372,99.455930,52.013058,6.202394,27.747552,46.463547,78.237214
2900207,Abaré (BA),8.125332,29.368030,94.960212,47.108753,77.347480,88.275862,61.591512,99.946950,99.098143,69.071618,28.488064,58.090186,56.498674,87.586207
2900306,Acajutiba (BA),21.273408,29.138577,94.839192,58.788332,70.456245,96.783844,94.016455,100.000000,99.925206,70.680628,2.692595,46.447270,60.433807,62.827225
2900355,Adustina (BA),22.930132,38.965836,78.547245,44.413666,87.196060,82.887042,76.546630,100.000000,100.000000,67.405355,12.526931,46.968298,47.122191,74.761465
2900405,Água Fria (BA),17.924965,36.729968,89.838337,55.334873,54.642032,88.036952,66.189376,100.000000,99.769053,56.212471,17.090069,23.279446,29.699769,61.847575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),33.221169,42.000612,92.822236,14.324985,64.645693,89.340257,81.795968,99.893097,99.679291,53.726329,20.983506,10.201588,50.717776,74.205864
2933406,Wagner (BA),26.490985,28.432732,88.397790,60.773481,74.033149,96.685083,84.806630,100.000000,100.000000,72.790055,1.104972,25.828729,56.491713,96.823204
2933455,Wanderley (BA),18.866787,24.231465,93.614458,8.132530,63.734940,83.012048,66.807229,99.397590,100.000000,55.963855,0.421687,17.469880,35.963855,67.108434


# Censo Demográfico

In [202]:
def formata_df(df):
    # extrai cod municipio
    df['cod_municipio']=df['Cod_setor'].apply(lambda x: str(x)[:7])
    # filtra setores rurais
    df=df[
        (df['Situacao_setor']>=4) & (df['Situacao_setor']<=8)]
    return df


def soma_e_agrupa_por_mun(df, cols=[])-> pd.DataFrame:
    name='var'
    df[name]=0
    for col in cols:
        df[col]=df[col].apply(lambda x: 0 if x=='X' else int(x))
        df[name]+=df[col]

    #agrupando por cidade
    df=df.groupby(['cod_municipio']).sum()   
    
    return df[[name]]

def calc_percentual(df, total, name):
    df=df.merge(total, left_index=True, right_index=True, how='outer')
    df[name]=df['var']/df['V001']*100
    return df[[name]]
    

Base para calcular percentual das demais variáveis
* Total de domicilios - Básico_BA V001 - Setores rurais 4 a 8

In [203]:
basico=pd.read_csv('Censo_Dem_BA_2010/Basico_BA.csv', sep=';',  encoding_errors='ignore')

In [204]:
print('Número de municípior na base: ', len(basico.Cod_municipio.unique()))

Número de municípior na base:  417


In [205]:
# SETORES RURAIS
basico=basico[
    (basico['Situacao_setor']>=4) & (basico['Situacao_setor']<=8)]

In [206]:
print('Número de municípior na base com setores rurais: ', len(basico.Cod_municipio.unique()))

Número de municípior na base com setores rurais:  415


Variável V001

In [207]:
total_dom_rurais=basico[['Cod_municipio', 'V001']].groupby(['Cod_municipio']).sum()
total_dom_rurais.index=total_dom_rurais.index.astype(str)

In [208]:
total_dom_rurais

,V001
Cod_municipio,
2900108,1381.0
2900207,1946.0
2900306,480.0
2900355,2831.0
2900405,2764.0
...,...
2933307,8543.0
2933406,638.0
2933455,1783.0


## Saúde

### Exposição a doenças advindas do esgoto a céu aberto  
- Somar V254, V252, V250 da tabela Entorno02_BA

In [209]:
entorno=pd.read_csv('Censo_Dem_BA_2010/Entorno02_BA.csv', sep=';', encoding_errors='ignore' )

In [210]:
entorno=formata_df(entorno)
entorno=soma_e_agrupa_por_mun(entorno, ['V254', 'V252', 'V250'])
entorno=calc_percentual(entorno, total_dom_rurais, 'Exposição a doenças advindas do esgoto a céu aberto')

In [211]:
base_final=base_final.merge(entorno[['Exposição a doenças advindas do esgoto a céu aberto']], left_index=True, right_index=True, how='outer')
base_final

,,Acesso à escola,Alfabetização na idade devida,Orientação técnica,Demais informações técnicas,Participação em grupos da sociedade civil,Acesso a crédito e financiamento,Acesso a outros programas do governo,Participação nos programas de pagamento por serviços ambientais,Participação no Programa Nacional de Habitação Rural,Acesso a aposentadoria ou pensão,Condição legal das terras,Eletricidade no domicílio,Comunicação via telefone,Acesso à internet,Exposição a doenças advindas do esgoto a céu aberto
cod_municipio,nome_municipio,,,,,,,,,,,,,,,
2900108,Abaíra (BA),12.078346,15.886834,96.517954,27.094668,76.496192,93.797606,69.205658,99.782372,99.455930,52.013058,6.202394,27.747552,46.463547,78.237214,0.0
2900207,Abaré (BA),8.125332,29.368030,94.960212,47.108753,77.347480,88.275862,61.591512,99.946950,99.098143,69.071618,28.488064,58.090186,56.498674,87.586207,0.0
2900306,Acajutiba (BA),21.273408,29.138577,94.839192,58.788332,70.456245,96.783844,94.016455,100.000000,99.925206,70.680628,2.692595,46.447270,60.433807,62.827225,0.0
2900355,Adustina (BA),22.930132,38.965836,78.547245,44.413666,87.196060,82.887042,76.546630,100.000000,100.000000,67.405355,12.526931,46.968298,47.122191,74.761465,0.0
2900405,Água Fria (BA),17.924965,36.729968,89.838337,55.334873,54.642032,88.036952,66.189376,100.000000,99.769053,56.212471,17.090069,23.279446,29.699769,61.847575,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),33.221169,42.000612,92.822236,14.324985,64.645693,89.340257,81.795968,99.893097,99.679291,53.726329,20.983506,10.201588,50.717776,74.205864,0.0
2933406,Wagner (BA),26.490985,28.432732,88.397790,60.773481,74.033149,96.685083,84.806630,100.000000,100.000000,72.790055,1.104972,25.828729,56.491713,96.823204,0.0
2933455,Wanderley (BA),18.866787,24.231465,93.614458,8.132530,63.734940,83.012048,66.807229,99.397590,100.000000,55.963855,0.421687,17.469880,35.963855,67.108434,0.0


## Registro Civil
- Pessoas de até 10 anos de idade que não tinham registro de nascimento: Pegar variável V002 da tabela Pessoa10_BA e ver percentual com o total de pessoas até 10 anos (soma de V024, V025, V026, V027, V028, V029, V030, V031, V032, V033, V034, V035, V036, V037, V038, V039, V040, V041, V042, V043, V044). 

In [212]:
pessoa=pd.read_csv('Censo_Dem_BA_2010/Pessoa10_BA.csv', sep=';', encoding_errors='ignore' )

In [213]:
pessoa=formata_df(pessoa)
pessoa=soma_e_agrupa_por_mun(pessoa, ['V002'])

In [214]:
pessoa_total=pd.read_csv('Censo_Dem_BA_2010/Pessoa13_BA.csv', sep=';', encoding_errors='ignore' )

In [215]:
pessoa_total=formata_df(pessoa_total)
pessoa_total=soma_e_agrupa_por_mun(
    pessoa_total, 
    ['V024', 'V025', 'V026', 'V027', 'V028', 'V029', 'V030', 'V031', 'V032', 'V033', 'V034', 'V035', 'V036', 'V037', 'V038', 'V039', 'V040', 'V041', 'V042', 'V043', 'V044'])

In [216]:
#percentual
pessoa=pessoa_total[['var']].merge(pessoa, left_index=True, right_index=True, how='outer')
pessoa['Registro Civil']=pessoa['var_y']/pessoa['var_x']*100

In [217]:
base_final=base_final.merge(pessoa[['Registro Civil']], left_index=True, right_index=True, how='outer')

In [218]:
base_final

,,Acesso à escola,Alfabetização na idade devida,Orientação técnica,Demais informações técnicas,Participação em grupos da sociedade civil,Acesso a crédito e financiamento,Acesso a outros programas do governo,Participação nos programas de pagamento por serviços ambientais,Participação no Programa Nacional de Habitação Rural,Acesso a aposentadoria ou pensão,Condição legal das terras,Eletricidade no domicílio,Comunicação via telefone,Acesso à internet,Exposição a doenças advindas do esgoto a céu aberto,Registro Civil
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),12.078346,15.886834,96.517954,27.094668,76.496192,93.797606,69.205658,99.782372,99.455930,52.013058,6.202394,27.747552,46.463547,78.237214,0.0,0.341297
2900207,Abaré (BA),8.125332,29.368030,94.960212,47.108753,77.347480,88.275862,61.591512,99.946950,99.098143,69.071618,28.488064,58.090186,56.498674,87.586207,0.0,0.227661
2900306,Acajutiba (BA),21.273408,29.138577,94.839192,58.788332,70.456245,96.783844,94.016455,100.000000,99.925206,70.680628,2.692595,46.447270,60.433807,62.827225,0.0,0.000000
2900355,Adustina (BA),22.930132,38.965836,78.547245,44.413666,87.196060,82.887042,76.546630,100.000000,100.000000,67.405355,12.526931,46.968298,47.122191,74.761465,0.0,0.297619
2900405,Água Fria (BA),17.924965,36.729968,89.838337,55.334873,54.642032,88.036952,66.189376,100.000000,99.769053,56.212471,17.090069,23.279446,29.699769,61.847575,0.0,0.110375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),33.221169,42.000612,92.822236,14.324985,64.645693,89.340257,81.795968,99.893097,99.679291,53.726329,20.983506,10.201588,50.717776,74.205864,0.0,0.382653
2933406,Wagner (BA),26.490985,28.432732,88.397790,60.773481,74.033149,96.685083,84.806630,100.000000,100.000000,72.790055,1.104972,25.828729,56.491713,96.823204,0.0,0.603622
2933455,Wanderley (BA),18.866787,24.231465,93.614458,8.132530,63.734940,83.012048,66.807229,99.397590,100.000000,55.963855,0.421687,17.469880,35.963855,67.108434,0.0,0.766871


### Capacidade de se alimentar e adquirir medicamentos
- Somar V005, V006, V007 da planilha DomicílioRenda e ver percentual com relação ao total de domicílios

In [219]:
dom_renda=pd.read_csv('Censo_Dem_BA_2010/DomicilioRenda_BA.csv', sep=';', encoding_errors='ignore')

In [220]:
dom_renda=formata_df(dom_renda)
dom_renda=soma_e_agrupa_por_mun(dom_renda, ['V005', 'V006', 'V007'])
dom_renda=calc_percentual(dom_renda, total_dom_rurais, 'Capacidade de se alimentar e adquirir medicamentos')

In [221]:
base_final=base_final.merge(dom_renda[['Capacidade de se alimentar e adquirir medicamentos']], left_index=True, right_index=True, how='outer')

In [222]:
base_final

,,Acesso à escola,Alfabetização na idade devida,Orientação técnica,Demais informações técnicas,Participação em grupos da sociedade civil,Acesso a crédito e financiamento,Acesso a outros programas do governo,Participação nos programas de pagamento por serviços ambientais,Participação no Programa Nacional de Habitação Rural,Acesso a aposentadoria ou pensão,Condição legal das terras,Eletricidade no domicílio,Comunicação via telefone,Acesso à internet,Exposição a doenças advindas do esgoto a céu aberto,Registro Civil,Capacidade de se alimentar e adquirir medicamentos
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),12.078346,15.886834,96.517954,27.094668,76.496192,93.797606,69.205658,99.782372,99.455930,52.013058,6.202394,27.747552,46.463547,78.237214,0.0,0.341297,52.860246
2900207,Abaré (BA),8.125332,29.368030,94.960212,47.108753,77.347480,88.275862,61.591512,99.946950,99.098143,69.071618,28.488064,58.090186,56.498674,87.586207,0.0,0.227661,63.566290
2900306,Acajutiba (BA),21.273408,29.138577,94.839192,58.788332,70.456245,96.783844,94.016455,100.000000,99.925206,70.680628,2.692595,46.447270,60.433807,62.827225,0.0,0.000000,68.125000
2900355,Adustina (BA),22.930132,38.965836,78.547245,44.413666,87.196060,82.887042,76.546630,100.000000,100.000000,67.405355,12.526931,46.968298,47.122191,74.761465,0.0,0.297619,66.125044
2900405,Água Fria (BA),17.924965,36.729968,89.838337,55.334873,54.642032,88.036952,66.189376,100.000000,99.769053,56.212471,17.090069,23.279446,29.699769,61.847575,0.0,0.110375,61.324168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),33.221169,42.000612,92.822236,14.324985,64.645693,89.340257,81.795968,99.893097,99.679291,53.726329,20.983506,10.201588,50.717776,74.205864,0.0,0.382653,63.783214
2933406,Wagner (BA),26.490985,28.432732,88.397790,60.773481,74.033149,96.685083,84.806630,100.000000,100.000000,72.790055,1.104972,25.828729,56.491713,96.823204,0.0,0.603622,69.592476
2933455,Wanderley (BA),18.866787,24.231465,93.614458,8.132530,63.734940,83.012048,66.807229,99.397590,100.000000,55.963855,0.421687,17.469880,35.963855,67.108434,0.0,0.766871,66.629277


### Iluminação pública
- Somar V209, V211, V213 de Entorno02 e verificar percentual com o número geral de domicílios rurais


In [223]:
entorno=pd.read_csv('Censo_Dem_BA_2010/Entorno02_BA.csv', sep=';', encoding_errors='ignore')

In [224]:
entorno=formata_df(entorno)
entorno=soma_e_agrupa_por_mun(entorno, ['V209', 'V211', 'V213'])
entorno=calc_percentual(entorno, total_dom_rurais, 'Iluminação pública')

In [225]:
base_final=base_final.merge(entorno[['Iluminação pública']], left_index=True, right_index=True, how='outer')
base_final

,,Acesso à escola,Alfabetização na idade devida,Orientação técnica,Demais informações técnicas,Participação em grupos da sociedade civil,Acesso a crédito e financiamento,Acesso a outros programas do governo,Participação nos programas de pagamento por serviços ambientais,Participação no Programa Nacional de Habitação Rural,Acesso a aposentadoria ou pensão,Condição legal das terras,Eletricidade no domicílio,Comunicação via telefone,Acesso à internet,Exposição a doenças advindas do esgoto a céu aberto,Registro Civil,Capacidade de se alimentar e adquirir medicamentos,Iluminação pública
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),12.078346,15.886834,96.517954,27.094668,76.496192,93.797606,69.205658,99.782372,99.455930,52.013058,6.202394,27.747552,46.463547,78.237214,0.0,0.341297,52.860246,0.0
2900207,Abaré (BA),8.125332,29.368030,94.960212,47.108753,77.347480,88.275862,61.591512,99.946950,99.098143,69.071618,28.488064,58.090186,56.498674,87.586207,0.0,0.227661,63.566290,0.0
2900306,Acajutiba (BA),21.273408,29.138577,94.839192,58.788332,70.456245,96.783844,94.016455,100.000000,99.925206,70.680628,2.692595,46.447270,60.433807,62.827225,0.0,0.000000,68.125000,0.0
2900355,Adustina (BA),22.930132,38.965836,78.547245,44.413666,87.196060,82.887042,76.546630,100.000000,100.000000,67.405355,12.526931,46.968298,47.122191,74.761465,0.0,0.297619,66.125044,0.0
2900405,Água Fria (BA),17.924965,36.729968,89.838337,55.334873,54.642032,88.036952,66.189376,100.000000,99.769053,56.212471,17.090069,23.279446,29.699769,61.847575,0.0,0.110375,61.324168,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),33.221169,42.000612,92.822236,14.324985,64.645693,89.340257,81.795968,99.893097,99.679291,53.726329,20.983506,10.201588,50.717776,74.205864,0.0,0.382653,63.783214,0.0
2933406,Wagner (BA),26.490985,28.432732,88.397790,60.773481,74.033149,96.685083,84.806630,100.000000,100.000000,72.790055,1.104972,25.828729,56.491713,96.823204,0.0,0.603622,69.592476,0.0
2933455,Wanderley (BA),18.866787,24.231465,93.614458,8.132530,63.734940,83.012048,66.807229,99.397590,100.000000,55.963855,0.421687,17.469880,35.963855,67.108434,0.0,0.766871,66.629277,0.0


### Pavimentação
- Somar V215, V217 e V219 em Entorno02 e verificar percentual com o número geral de domicílios rurais.

In [226]:
entorno=pd.read_csv('Censo_Dem_BA_2010/Entorno02_BA.csv', sep=';', encoding_errors='ignore')

In [227]:
entorno=formata_df(entorno)
entorno=soma_e_agrupa_por_mun(entorno, ['V215', 'V217','V219'])
entorno=calc_percentual(entorno, total_dom_rurais, 'Pavimentação')

In [228]:
base_final=base_final.merge(entorno[['Pavimentação']], left_index=True, right_index=True, how='outer')
base_final

,,Acesso à escola,Alfabetização na idade devida,Orientação técnica,Demais informações técnicas,Participação em grupos da sociedade civil,Acesso a crédito e financiamento,Acesso a outros programas do governo,Participação nos programas de pagamento por serviços ambientais,Participação no Programa Nacional de Habitação Rural,Acesso a aposentadoria ou pensão,Condição legal das terras,Eletricidade no domicílio,Comunicação via telefone,Acesso à internet,Exposição a doenças advindas do esgoto a céu aberto,Registro Civil,Capacidade de se alimentar e adquirir medicamentos,Iluminação pública,Pavimentação
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),12.078346,15.886834,96.517954,27.094668,76.496192,93.797606,69.205658,99.782372,99.455930,52.013058,6.202394,27.747552,46.463547,78.237214,0.0,0.341297,52.860246,0.0,0.0
2900207,Abaré (BA),8.125332,29.368030,94.960212,47.108753,77.347480,88.275862,61.591512,99.946950,99.098143,69.071618,28.488064,58.090186,56.498674,87.586207,0.0,0.227661,63.566290,0.0,0.0
2900306,Acajutiba (BA),21.273408,29.138577,94.839192,58.788332,70.456245,96.783844,94.016455,100.000000,99.925206,70.680628,2.692595,46.447270,60.433807,62.827225,0.0,0.000000,68.125000,0.0,0.0
2900355,Adustina (BA),22.930132,38.965836,78.547245,44.413666,87.196060,82.887042,76.546630,100.000000,100.000000,67.405355,12.526931,46.968298,47.122191,74.761465,0.0,0.297619,66.125044,0.0,0.0
2900405,Água Fria (BA),17.924965,36.729968,89.838337,55.334873,54.642032,88.036952,66.189376,100.000000,99.769053,56.212471,17.090069,23.279446,29.699769,61.847575,0.0,0.110375,61.324168,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),33.221169,42.000612,92.822236,14.324985,64.645693,89.340257,81.795968,99.893097,99.679291,53.726329,20.983506,10.201588,50.717776,74.205864,0.0,0.382653,63.783214,0.0,0.0
2933406,Wagner (BA),26.490985,28.432732,88.397790,60.773481,74.033149,96.685083,84.806630,100.000000,100.000000,72.790055,1.104972,25.828729,56.491713,96.823204,0.0,0.603622,69.592476,0.0,0.0
2933455,Wanderley (BA),18.866787,24.231465,93.614458,8.132530,63.734940,83.012048,66.807229,99.397590,100.000000,55.963855,0.421687,17.469880,35.963855,67.108434,0.0,0.766871,66.629277,0.0,0.0


### Aquisição de ativos
- Somar V005, V006, V007, V008 da DomicílioRenda (renda domiciliar abaixo de 1 salário-mínimo) e ver percentual com o total de domicílios. 
Somar todas as receitas, selecionar apenas os domicílios com rendimento abaixo de R$ 945,50 e ver percentual com total de domicílios. 

In [229]:
dom_renda=pd.read_csv('Censo_Dem_BA_2010/DomicilioRenda_BA.csv', sep=';', encoding_errors='ignore')

In [230]:
dom_renda=formata_df(dom_renda)
dom_renda=soma_e_agrupa_por_mun(dom_renda, ['V005', 'V006', 'V007', 'V008'])
dom_renda=calc_percentual(dom_renda, total_dom_rurais, 'Aquisição de ativos')

In [231]:
base_final=base_final.merge(dom_renda[['Aquisição de ativos']], left_index=True, right_index=True, how='outer')

In [232]:
base_final

,,Acesso à escola,Alfabetização na idade devida,Orientação técnica,Demais informações técnicas,Participação em grupos da sociedade civil,Acesso a crédito e financiamento,Acesso a outros programas do governo,Participação nos programas de pagamento por serviços ambientais,Participação no Programa Nacional de Habitação Rural,Acesso a aposentadoria ou pensão,Condição legal das terras,Eletricidade no domicílio,Comunicação via telefone,Acesso à internet,Exposição a doenças advindas do esgoto a céu aberto,Registro Civil,Capacidade de se alimentar e adquirir medicamentos,Iluminação pública,Pavimentação,Aquisição de ativos
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),12.078346,15.886834,96.517954,27.094668,76.496192,93.797606,69.205658,99.782372,99.455930,52.013058,6.202394,27.747552,46.463547,78.237214,0.0,0.341297,52.860246,0.0,0.0,88.486604
2900207,Abaré (BA),8.125332,29.368030,94.960212,47.108753,77.347480,88.275862,61.591512,99.946950,99.098143,69.071618,28.488064,58.090186,56.498674,87.586207,0.0,0.227661,63.566290,0.0,0.0,80.318602
2900306,Acajutiba (BA),21.273408,29.138577,94.839192,58.788332,70.456245,96.783844,94.016455,100.000000,99.925206,70.680628,2.692595,46.447270,60.433807,62.827225,0.0,0.000000,68.125000,0.0,0.0,87.500000
2900355,Adustina (BA),22.930132,38.965836,78.547245,44.413666,87.196060,82.887042,76.546630,100.000000,100.000000,67.405355,12.526931,46.968298,47.122191,74.761465,0.0,0.297619,66.125044,0.0,0.0,87.389615
2900405,Água Fria (BA),17.924965,36.729968,89.838337,55.334873,54.642032,88.036952,66.189376,100.000000,99.769053,56.212471,17.090069,23.279446,29.699769,61.847575,0.0,0.110375,61.324168,0.0,0.0,82.706223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),33.221169,42.000612,92.822236,14.324985,64.645693,89.340257,81.795968,99.893097,99.679291,53.726329,20.983506,10.201588,50.717776,74.205864,0.0,0.382653,63.783214,0.0,0.0,88.587147
2933406,Wagner (BA),26.490985,28.432732,88.397790,60.773481,74.033149,96.685083,84.806630,100.000000,100.000000,72.790055,1.104972,25.828729,56.491713,96.823204,0.0,0.603622,69.592476,0.0,0.0,90.438871
2933455,Wanderley (BA),18.866787,24.231465,93.614458,8.132530,63.734940,83.012048,66.807229,99.397590,100.000000,55.963855,0.421687,17.469880,35.963855,67.108434,0.0,0.766871,66.629277,0.0,0.0,89.568144


### Coleta de lixo
- Somar variáveis V038, V039, V040, V041 e V042 tabela Domicílio01_BA e fazer o percentual com o número geral de domicílios rurais

In [233]:
dom_01=pd.read_csv('Censo_Dem_BA_2010/Domicilio01_BA.csv', sep=';', encoding_errors='ignore')

In [234]:
dom_01=formata_df(dom_01)
dom_01=soma_e_agrupa_por_mun(dom_01, ['V038', 'V039', 'V040', 'V041', 'V042'])
dom_01=calc_percentual(dom_01, total_dom_rurais, 'Coleta de lixo')

In [235]:
base_final=base_final.merge(dom_01[['Coleta de lixo']], left_index=True, right_index=True, how='outer')

In [236]:
base_final

,,Acesso à escola,Alfabetização na idade devida,Orientação técnica,Demais informações técnicas,Participação em grupos da sociedade civil,Acesso a crédito e financiamento,Acesso a outros programas do governo,Participação nos programas de pagamento por serviços ambientais,Participação no Programa Nacional de Habitação Rural,Acesso a aposentadoria ou pensão,...,Eletricidade no domicílio,Comunicação via telefone,Acesso à internet,Exposição a doenças advindas do esgoto a céu aberto,Registro Civil,Capacidade de se alimentar e adquirir medicamentos,Iluminação pública,Pavimentação,Aquisição de ativos,Coleta de lixo
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),12.078346,15.886834,96.517954,27.094668,76.496192,93.797606,69.205658,99.782372,99.455930,52.013058,...,27.747552,46.463547,78.237214,0.0,0.341297,52.860246,0.0,0.0,88.486604,80.159305
2900207,Abaré (BA),8.125332,29.368030,94.960212,47.108753,77.347480,88.275862,61.591512,99.946950,99.098143,69.071618,...,58.090186,56.498674,87.586207,0.0,0.227661,63.566290,0.0,0.0,80.318602,98.355601
2900306,Acajutiba (BA),21.273408,29.138577,94.839192,58.788332,70.456245,96.783844,94.016455,100.000000,99.925206,70.680628,...,46.447270,60.433807,62.827225,0.0,0.000000,68.125000,0.0,0.0,87.500000,95.000000
2900355,Adustina (BA),22.930132,38.965836,78.547245,44.413666,87.196060,82.887042,76.546630,100.000000,100.000000,67.405355,...,46.968298,47.122191,74.761465,0.0,0.297619,66.125044,0.0,0.0,87.389615,85.694101
2900405,Água Fria (BA),17.924965,36.729968,89.838337,55.334873,54.642032,88.036952,66.189376,100.000000,99.769053,56.212471,...,23.279446,29.699769,61.847575,0.0,0.110375,61.324168,0.0,0.0,82.706223,88.748191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),33.221169,42.000612,92.822236,14.324985,64.645693,89.340257,81.795968,99.893097,99.679291,53.726329,...,10.201588,50.717776,74.205864,0.0,0.382653,63.783214,0.0,0.0,88.587147,83.038745
2933406,Wagner (BA),26.490985,28.432732,88.397790,60.773481,74.033149,96.685083,84.806630,100.000000,100.000000,72.790055,...,25.828729,56.491713,96.823204,0.0,0.603622,69.592476,0.0,0.0,90.438871,81.974922
2933455,Wanderley (BA),18.866787,24.231465,93.614458,8.132530,63.734940,83.012048,66.807229,99.397590,100.000000,55.963855,...,17.469880,35.963855,67.108434,0.0,0.766871,66.629277,0.0,0.0,89.568144,92.708918


### Esgotamento sanitário
- Somar variáveis V019, V020, V021, V022, e V023 da tabela Domicílio01_BA e fazer o percentual com o número geral de domicílios rurais

In [237]:
dom_01=pd.read_csv('Censo_Dem_BA_2010/Domicilio01_BA.csv', sep=';', encoding_errors='ignore')

In [238]:
dom_01=formata_df(dom_01)
dom_01=soma_e_agrupa_por_mun(dom_01, ['V019', 'V020', 'V021', 'V022','V023'])
dom_01=calc_percentual(dom_01, total_dom_rurais, 'Esgotamento sanitário')

In [239]:
base_final=base_final.merge(dom_01[['Esgotamento sanitário']], left_index=True, right_index=True, how='outer')
base_final

,,Acesso à escola,Alfabetização na idade devida,Orientação técnica,Demais informações técnicas,Participação em grupos da sociedade civil,Acesso a crédito e financiamento,Acesso a outros programas do governo,Participação nos programas de pagamento por serviços ambientais,Participação no Programa Nacional de Habitação Rural,Acesso a aposentadoria ou pensão,...,Comunicação via telefone,Acesso à internet,Exposição a doenças advindas do esgoto a céu aberto,Registro Civil,Capacidade de se alimentar e adquirir medicamentos,Iluminação pública,Pavimentação,Aquisição de ativos,Coleta de lixo,Esgotamento sanitário
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),12.078346,15.886834,96.517954,27.094668,76.496192,93.797606,69.205658,99.782372,99.455930,52.013058,...,46.463547,78.237214,0.0,0.341297,52.860246,0.0,0.0,88.486604,80.159305,97.320782
2900207,Abaré (BA),8.125332,29.368030,94.960212,47.108753,77.347480,88.275862,61.591512,99.946950,99.098143,69.071618,...,56.498674,87.586207,0.0,0.227661,63.566290,0.0,0.0,80.318602,98.355601,93.371017
2900306,Acajutiba (BA),21.273408,29.138577,94.839192,58.788332,70.456245,96.783844,94.016455,100.000000,99.925206,70.680628,...,60.433807,62.827225,0.0,0.000000,68.125000,0.0,0.0,87.500000,95.000000,95.208333
2900355,Adustina (BA),22.930132,38.965836,78.547245,44.413666,87.196060,82.887042,76.546630,100.000000,100.000000,67.405355,...,47.122191,74.761465,0.0,0.297619,66.125044,0.0,0.0,87.389615,85.694101,97.562699
2900405,Água Fria (BA),17.924965,36.729968,89.838337,55.334873,54.642032,88.036952,66.189376,100.000000,99.769053,56.212471,...,29.699769,61.847575,0.0,0.110375,61.324168,0.0,0.0,82.706223,88.748191,99.891462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),33.221169,42.000612,92.822236,14.324985,64.645693,89.340257,81.795968,99.893097,99.679291,53.726329,...,50.717776,74.205864,0.0,0.382653,63.783214,0.0,0.0,88.587147,83.038745,87.861407
2933406,Wagner (BA),26.490985,28.432732,88.397790,60.773481,74.033149,96.685083,84.806630,100.000000,100.000000,72.790055,...,56.491713,96.823204,0.0,0.603622,69.592476,0.0,0.0,90.438871,81.974922,99.686520
2933455,Wanderley (BA),18.866787,24.231465,93.614458,8.132530,63.734940,83.012048,66.807229,99.397590,100.000000,55.963855,...,35.963855,67.108434,0.0,0.766871,66.629277,0.0,0.0,89.568144,92.708918,98.485698


### Abastecimento de água canalizada
- Somar variáveis V013, V014 e V015 da tabela Domicílio01_BA e fazer o percentual com o número geral de domicílios rurais

In [240]:
dom_01=pd.read_csv('Censo_Dem_BA_2010/Domicilio01_BA.csv', sep=';', encoding_errors='ignore')

In [241]:
dom_01=formata_df(dom_01)
dom_01=soma_e_agrupa_por_mun(dom_01, ['V013', 'V014', 'V015'])
dom_01=calc_percentual(dom_01, total_dom_rurais, 'Abastecimento de água canalizada')

In [242]:
base_final=base_final.merge(dom_01[['Abastecimento de água canalizada']], left_index=True, right_index=True, how='outer')
base_final

,,Acesso à escola,Alfabetização na idade devida,Orientação técnica,Demais informações técnicas,Participação em grupos da sociedade civil,Acesso a crédito e financiamento,Acesso a outros programas do governo,Participação nos programas de pagamento por serviços ambientais,Participação no Programa Nacional de Habitação Rural,Acesso a aposentadoria ou pensão,...,Acesso à internet,Exposição a doenças advindas do esgoto a céu aberto,Registro Civil,Capacidade de se alimentar e adquirir medicamentos,Iluminação pública,Pavimentação,Aquisição de ativos,Coleta de lixo,Esgotamento sanitário,Abastecimento de água canalizada
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),12.078346,15.886834,96.517954,27.094668,76.496192,93.797606,69.205658,99.782372,99.455930,52.013058,...,78.237214,0.0,0.341297,52.860246,0.0,0.0,88.486604,80.159305,97.320782,15.061550
2900207,Abaré (BA),8.125332,29.368030,94.960212,47.108753,77.347480,88.275862,61.591512,99.946950,99.098143,69.071618,...,87.586207,0.0,0.227661,63.566290,0.0,0.0,80.318602,98.355601,93.371017,49.537513
2900306,Acajutiba (BA),21.273408,29.138577,94.839192,58.788332,70.456245,96.783844,94.016455,100.000000,99.925206,70.680628,...,62.827225,0.0,0.000000,68.125000,0.0,0.0,87.500000,95.000000,95.208333,77.916667
2900355,Adustina (BA),22.930132,38.965836,78.547245,44.413666,87.196060,82.887042,76.546630,100.000000,100.000000,67.405355,...,74.761465,0.0,0.297619,66.125044,0.0,0.0,87.389615,85.694101,97.562699,73.507594
2900405,Água Fria (BA),17.924965,36.729968,89.838337,55.334873,54.642032,88.036952,66.189376,100.000000,99.769053,56.212471,...,61.847575,0.0,0.110375,61.324168,0.0,0.0,82.706223,88.748191,99.891462,75.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),33.221169,42.000612,92.822236,14.324985,64.645693,89.340257,81.795968,99.893097,99.679291,53.726329,...,74.205864,0.0,0.382653,63.783214,0.0,0.0,88.587147,83.038745,87.861407,80.369893
2933406,Wagner (BA),26.490985,28.432732,88.397790,60.773481,74.033149,96.685083,84.806630,100.000000,100.000000,72.790055,...,96.823204,0.0,0.603622,69.592476,0.0,0.0,90.438871,81.974922,99.686520,52.664577
2933455,Wanderley (BA),18.866787,24.231465,93.614458,8.132530,63.734940,83.012048,66.807229,99.397590,100.000000,55.963855,...,67.108434,0.0,0.766871,66.629277,0.0,0.0,89.568144,92.708918,98.485698,64.890634


### DataSUS

No datasus, o codigo do municipio em 6 dígitos, os 6 primeiros das bases anteriores. Portanto fiz o mapeamento.

Para algumas combinações de variáveis nao houveram dados para o ano em questão

In [243]:
cidades_ba['id_6']=cidades_ba['id'].apply(lambda x: x[:6])

<ipython-input-243-61191858d4c8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cidades_ba['id_6']=cidades_ba['id'].apply(lambda x: x[:6])


In [244]:
aux1=pd.read_csv('datasus/agrotoxico_acidental_2017_2.csv', sep=';')

aux1=aux1[:-1] #ultima linha é total, nao precisa

#extrai cód de 6 dígitos
aux1['cod_municipio_6']=aux1['Município de residência'].apply(lambda x: x[:6])
aux1=aux1[['cod_municipio_6', 'Total']]

#merge com códigos corretos
aux1=cidades_ba[['id', 'id_6']].merge(aux1, left_on='id_6', right_on='cod_municipio_6')
aux1=aux1[['id', 'Total']]
aux1.set_index('id', inplace=True)

In [245]:
aux2=pd.read_csv('datasus/agrotoxico_uso_habitual_2017_2.csv', sep=';')

aux2=aux2[:-1] #ultima linha é total, nao precisa

#extrai cód de 6 dígitos
aux2['cod_municipio_6']=aux2['Município de residência'].apply(lambda x: x[:6])
aux2=aux2[['cod_municipio_6', 'Total']]

#merge com códigos corretos
aux2=cidades_ba[['id', 'id_6']].merge(aux2, left_on='id_6', right_on='cod_municipio_6')
aux2=aux2[['id', 'Total']]
aux2.set_index('id', inplace=True)

In [246]:
aux3=pd.read_csv('datasus/agrotoxico_erro_adm_2017_2.csv', sep=';')

aux3=aux3[:-1] #ultima linha é total, nao precisa

#extrai cód de 6 dígitos
aux3['cod_municipio_6']=aux3['Município de residência'].apply(lambda x: x[:6])
aux3=aux3[['cod_municipio_6', 'Total']]

#merge com códigos corretos
aux3=cidades_ba[['id', 'id_6']].merge(aux3, left_on='id_6', right_on='cod_municipio_6')
aux3=aux3[['id', 'Total']]
aux3.set_index('id', inplace=True)

In [247]:
aux4=pd.read_csv('datasus/veterinario_acidental_2017_2.csv', sep=';')

aux4=aux4[:-1] #ultima linha é total, nao precisa

#extrai cód de 6 dígitos
aux4['cod_municipio_6']=aux4['Município de residência'].apply(lambda x: x[:6])
aux4=aux4[['cod_municipio_6', 'Total']]

#merge com códigos corretos
aux4=cidades_ba[['id', 'id_6']].merge(aux4, left_on='id_6', right_on='cod_municipio_6')
aux4=aux4[['id', 'Total']]
aux4.set_index('id', inplace=True)

In [248]:
# somando todos os dados
aux=pd.concat([aux1, aux2, aux3, aux4], axis=1)
aux.fillna(0, inplace=True)

aux=aux.sum(axis=1)
aux=aux.to_frame()

In [249]:
aux.rename(columns={0: 'Contaminação do trabalhador rural'}, inplace=True)
aux.index.names=['cod_municipio']

calculando porcentagem sobre população rural

In [250]:
pessoa=pd.read_csv('Censo_Dem_BA_2010/Pessoa03_BA.csv', sep=';', encoding_errors='ignore')

In [251]:
pessoa=formata_df(pessoa)
pessoa=soma_e_agrupa_por_mun(pessoa, ['V001'])
aux=pessoa.merge(aux, left_index=True, right_index=True)

In [252]:
aux['Contaminação do trabalhador rural']=aux['Contaminação do trabalhador rural']/aux['var']*100
aux=aux[['Contaminação do trabalhador rural']]

Merge com base final

In [253]:
base_final=base_final.merge(aux, right_index=True, left_index=True, how='outer')
base_final.fillna(0, inplace=True)

In [254]:
base_final

,,Acesso à escola,Alfabetização na idade devida,Orientação técnica,Demais informações técnicas,Participação em grupos da sociedade civil,Acesso a crédito e financiamento,Acesso a outros programas do governo,Participação nos programas de pagamento por serviços ambientais,Participação no Programa Nacional de Habitação Rural,Acesso a aposentadoria ou pensão,...,Exposição a doenças advindas do esgoto a céu aberto,Registro Civil,Capacidade de se alimentar e adquirir medicamentos,Iluminação pública,Pavimentação,Aquisição de ativos,Coleta de lixo,Esgotamento sanitário,Abastecimento de água canalizada,Contaminação do trabalhador rural
cod_municipio,nome_municipio,,,,,,,,,,,,,,,,,,,,,
2900108,Abaíra (BA),12.078346,15.886834,96.517954,27.094668,76.496192,93.797606,69.205658,99.782372,99.455930,52.013058,...,0.0,0.341297,52.860246,0.0,0.0,88.486604,80.159305,97.320782,15.061550,0.000000
2900207,Abaré (BA),8.125332,29.368030,94.960212,47.108753,77.347480,88.275862,61.591512,99.946950,99.098143,69.071618,...,0.0,0.227661,63.566290,0.0,0.0,80.318602,98.355601,93.371017,49.537513,0.024891
2900306,Acajutiba (BA),21.273408,29.138577,94.839192,58.788332,70.456245,96.783844,94.016455,100.000000,99.925206,70.680628,...,0.0,0.000000,68.125000,0.0,0.0,87.500000,95.000000,95.208333,77.916667,0.000000
2900355,Adustina (BA),22.930132,38.965836,78.547245,44.413666,87.196060,82.887042,76.546630,100.000000,100.000000,67.405355,...,0.0,0.297619,66.125044,0.0,0.0,87.389615,85.694101,97.562699,73.507594,0.000000
2900405,Água Fria (BA),17.924965,36.729968,89.838337,55.334873,54.642032,88.036952,66.189376,100.000000,99.769053,56.212471,...,0.0,0.110375,61.324168,0.0,0.0,82.706223,88.748191,99.891462,75.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933307,Vitória da Conquista (BA),33.221169,42.000612,92.822236,14.324985,64.645693,89.340257,81.795968,99.893097,99.679291,53.726329,...,0.0,0.382653,63.783214,0.0,0.0,88.587147,83.038745,87.861407,80.369893,0.000000
2933406,Wagner (BA),26.490985,28.432732,88.397790,60.773481,74.033149,96.685083,84.806630,100.000000,100.000000,72.790055,...,0.0,0.603622,69.592476,0.0,0.0,90.438871,81.974922,99.686520,52.664577,0.000000
2933455,Wanderley (BA),18.866787,24.231465,93.614458,8.132530,63.734940,83.012048,66.807229,99.397590,100.000000,55.963855,...,0.0,0.766871,66.629277,0.0,0.0,89.568144,92.708918,98.485698,64.890634,0.000000


In [255]:
base_final.to_csv('Dados.csv')
base_final.to_csv('Dados_windows.csv', sep=';', decimal=',')

#### checando se há valores fora do padrão

In [256]:
for col in base_final.columns:
    if not base_final[base_final[col]>100].empty:
        print(col)
        print(base_final[base_final[col]>100])

In [257]:
for col in base_final.columns:
    if not base_final[base_final[col]<0].empty:
        print(col)
        print(base_final[base_final[col]<0])